## Описание задачи

Разбираем в ноутбуке стандартный CGCNN из репозитория. Смотрим как он работает на датасете из CIF файлов. Этот датасет был использован мной при запуске CGCNN через командную строку (эта задача с использованием датасета была описана в статье https://dzen.ru/a/Z6Jxn-2D4QCDm3vj )  

В данном проекте стояла задача разабраться в коде проекта Американского университета MIT https://github.com/NikLaz25/GNN_for_materials_1/tree/main  Соответственно взят из код с незначительными корректировками для его использования в отдельном файле jupyter notebook. Посколько задача была разобраться в файле содержится множество описаний кода и распечаток содержания структуры разных объектов.

Файл **[main.py](https://github.com/txie-93/cgcnn/blob/master/main.py)** из репозитория **CGCNN (Crystal Graph Convolutional Neural Networks)** — это основной скрипт для обучения и оценки модели, предсказывающей свойства кристаллических материалов. Разберём его структуру и функциональность:

---

##### **1. Назначение файла**
Главные задачи `main.py`:
- **Обучение модели CGCNN** на данных кристаллических структур (в формате CIF-файлов).
- **Оценка качества модели** на валидационной и тестовой выборках.
- **Сохранение чекпоинтов** лучших версий модели.
- **Интеграция с командной строкой** для гибкой настройки параметров через аргументы.

---

##### **2. Ключевые инструменты и их роль**
###### **Аргументы командной строки (argparse)**
Файл принимает параметры через `argparse`, что позволяет настраивать:
- **Пути к данным**: `root_dir` с CIF-файлами и `id_prop.csv` .
- **Гиперпараметры модели**: 
  - `--atom-fea-len`: размер атомных эмбеддингов.
  - `--h-fea-len`: размер скрытых слоёв после пулинга.
  - `--n-conv`: количество графовых свёрточных слоёв .
- **Параметры обучения**:
  - `--lr`: скорость обучения.
  - `--batch-size`: размер батча.
  - `--epochs`: число эпох.

###### **Загрузка данных (CIFData)**
- Класс `CIFData` из `cgcnn.data` загружает кристаллические структуры из CIF-файлов и их свойства из `id_prop.csv` .
- Использует `collate_pool` для объединения данных в батчи с учётом разного числа атомов в кристаллах.

###### **Модель (CrystalGraphConvNet)**
- **Архитектура CGCNN**:
  - **Графовые свёртки**: Преобразуют атомные признаки с учётом связей между соседями.
  - **Пулинг (global_mean_pool)**: Агрегирует атомные эмбеддинги в дескриптор кристалла.
  - **Полносвязные слои**: Предсказывают целевое свойство (например, энергию образования) .

###### **Оптимизация и обучение**
- **Оптимизатор**: По умолчанию — SGD, но можно выбрать Adam (`--optim`).
- **Планировщик LR**: `MultiStepLR` для изменения learning rate на заданных эпохах (`--lr-milestones`).
- **Функция потерь**: Для регрессии — MSE, для классификации — кросс-энтропия .

###### **Логирование и сохранение**
- **Вывод метрик**: Средняя абсолютная ошибка (MAE) для регрессии, точность для классификации.
- **Чекпоинты**: Сохраняются в `model_best.pth.tar` при улучшении качества на валидации.

---

##### **3. Пример запуска**
```bash
python main.py ./data --task regression --epochs 100 --batch-size 32
```
Где:
- `./data` — папка с CIF-файлами и `id_prop.csv`.
- `--task regression` — режим регрессии (предсказание энергии).

---

##### **4. Важные особенности**
1. **Гибкость входных данных**:
   - Поддерживает как регрессию, так и классификацию (`--task`).
   - Позволяет задавать размер выборок через `--train-ratio`/`--train-size` .

2. **Интеграция с PyTorch**:
   - Использует `DataLoader` для эффективной загрузки данных.
   - Поддержка GPU (`--disable-cuda` для отключения).

3. **Воспроизводимость**:
   - Фиксирует seed для случайных процессов.
   - Сохраняет лучшую модель для последующего использования .

---

##### **5. Сравнение с аналогами**
CGCNN отличается от других GNN-архитектур для материалов:
- **Упрощённый подход**: Не учитывает углы между связями (в отличие от DimeNet++).
- **Акцент на кристаллическую структуру**: Использует параметры решётки и межатомные расстояния .

---

##### **Заключение**
`main.py` в CGCNN — это **готовый пайплайн** для обучения модели на кристаллических данных. Он сочетает:
- Загрузку и предобработку CIF-файлов.
- Графовые свёртки для анализа локального окружения атомов.
- Гибкую настройку через командную строку.

Для более сложных задач (например, предсказания нескольких свойств) можно рассмотреть **mt-cgcnn** (мультитаргетную версию) .

В файле **[model.py](https://github.com/txie-93/cgcnn/blob/master/cgcnn/model.py)** реализована архитектура **Crystal Graph Convolutional Neural Network (CGCNN)** — специализированной графовой нейросети для предсказания свойств кристаллических материалов. Разберём код с подробными комментариями:

---

##### **1. Импорты**
```python
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch_geometric.nn import global_mean_pool  # Для агрегации атомных признаков
```
- **torch.nn**: Базовые модули PyTorch (линейные слои, функции активации).
- **global_mean_pool**: Пулинг-слой для объединения атомных признаков в графовый дескриптор.

---

##### **2. Класс `CrystalGraphConvNet`**
Главный класс модели, наследуемый от `nn.Module`.

###### **Инициализация (`__init__`)**
```python
def __init__(self, orig_atom_fea_len, nbr_fea_len, atom_fea_len=64, n_conv=3, h_fea_len=128, n_h=1):
    super(CrystalGraphConvNet, self).__init__()
```
- **Параметры**:
  - `orig_atom_fea_len`: Размер исходных атомных признаков (например, 92 для атомного номера).
  - `nbr_fea_len`: Размер признаков связи (например, межатомное расстояние).
  - `atom_fea_len`: Размер скрытых атомных эмбеддингов (по умолчанию 64).
  - `n_conv`: Количество графовых свёрточных слоёв (по умолчанию 3).
  - `h_fea_len`: Размер скрытого слоя после пулинга (по умолчанию 128).
  - `n_h`: Количество полносвязных слоёв (по умолчанию 1).

---

##### **3. Графовые свёрточные слои (`conv_layers`)**
```python
self.conv_layers = nn.ModuleList([
    ConvLayer(orig_atom_fea_len=orig_atom_fea_len if i == 0 else atom_fea_len,
              nbr_fea_len=nbr_fea_len,
              atom_fea_len=atom_fea_len)
    for i in range(n_conv)
])
```
- **ConvLayer**: Кастомный свёрточный слой (определён ниже).
- **Особенности**:
  - Первый слой принимает исходные атомные признаки (`orig_atom_fea_len`).
  - Последующие слои работают с выходом предыдущих (`atom_fea_len`).
  - Число слоёв задаётся через `n_conv`.

---

##### **4. Полносвязные слои (`fc_layers`)**
```python
self.fc_layers = nn.ModuleList([
    nn.Linear(atom_fea_len if i == 0 else h_fea_len, 
              h_fea_len)
    for i in range(n_h)
])
```
- **Структура**:
  - Линейные слои с размерностью `atom_fea_len → h_fea_len` (если `n_h=1`, то один слой).
  - Активация `ReLU` применяется после каждого слоя.

---

##### **5. Выходной слой (`fc_out`)**
```python
self.fc_out = nn.Linear(h_fea_len, 1)  # Регрессия (1 выход)
```
- Предсказывает **скалярное значение** (например, энергию образования).

---

##### **6. Прямой проход (`forward`)**
```python
def forward(self, atom_fea, nbr_fea, nbr_idx, crystal_atom_idx):
    # 1. Графовые свёртки
    for conv in self.conv_layers:
        atom_fea = conv(atom_fea, nbr_fea, nbr_idx)
    
    # 2. Глобальный пулинг
    crys_fea = global_mean_pool(atom_fea, crystal_atom_idx)
    
    # 3. Полносвязные слои
    for fc in self.fc_layers:
        crys_fea = torch.relu(fc(crys_fea))
    
    # 4. Выход
    return self.fc_out(crys_fea)
```
- **Входы**:
  - `atom_fea`: Признаки атомов `[num_atoms, orig_atom_fea_len]`.
  - `nbr_fea`: Признаки связей `[num_atoms, max_nbr, nbr_fea_len]`.
  - `nbr_idx`: Индексы соседей `[num_atoms, max_nbr]`.
  - `crystal_atom_idx`: Индексы атомов для пулинга `[num_atoms]`.
- **Шаги**:
  1. **Графовые свёртки**: Обновляют атомные признаки с учётом локального окружения.
  2. **Пулинг**: Усреднение признаков по атому для всего кристалла.
  3. **Полносвязные слои**: Преобразуют глобальные признаки.
  4. **Выход**: Предсказание свойства.

---

##### **7. Класс `ConvLayer` (графовая свёртка)**
```python
class ConvLayer(nn.Module):
    def __init__(self, orig_atom_fea_len, nbr_fea_len, atom_fea_len):
        super(ConvLayer, self).__init__()
        self.fc_full = nn.Linear(2*orig_atom_fea_len + nbr_fea_len, atom_fea_len)
        self.sigmoid = nn.Sigmoid()
        self.softplus = nn.Softplus()
```
- **Формула свёртки**:
  ```
  h_i^(l+1) = h_i^l + Σ_{j∈N(i)} σ(W_f * [h_i^l || h_j^l || e_ij] + b_f)
  ```
  Где:
  - `h_i^l`: Признаки атома `i` на слое `l`.
  - `e_ij`: Признаки связи между атомами `i` и `j`.
  - `σ`: Сигмоидальная активация для "ворот" (gate mechanism).

---

##### **8. Прямой проход в `ConvLayer`**
```python
def forward(self, atom_in_fea, nbr_fea, nbr_idx):
    # 1. Сбор признаков соседей
    N, M = nbr_idx.shape  # N = число атомов, M = число соседей
    atom_nbr_fea = atom_in_fea[nbr_idx.view(-1)].view(N, M, -1)
    
    # 2. Конкатенация признаков
    total_fea = torch.cat([atom_in_fea.unsqueeze(1).expand(N, M, -1),
                           atom_nbr_fea,
                           nbr_fea], dim=2)
    
    # 3. Применение линейного слоя и активаций
    total_fea = self.softplus(self.fc_full(total_fea))
    total_fea = self.sigmoid(total_fea)
    
    # 4. Агрегация по соседям
    return torch.sum(total_fea, dim=1)
```
- **Результат**: Обновлённые атомные признаки `[num_atoms, atom_fea_len]`.

---

##### **9. Особенности архитектуры**
1. **Учёт локального окружения**:
   - Графовые свёртки используют информацию о соседних атомах и межатомных расстояниях.
2. **Простота**:
   - Нет учёта углов между связями (в отличие от DimeNet++).
3. **Эффективность**:
   - Пулинг позволяет обрабатывать кристаллы с разным числом атомов.

---

##### **10. Пример использования**
```python
model = CrystalGraphConvNet(orig_atom_fea_len=92, nbr_fea_len=5)
output = model(atom_fea, nbr_fea, nbr_idx, crystal_atom_idx)  # [batch_size, 1]
```

Этот файл — **ядро CGCNN**, реализующее ключевые механизмы для работы с кристаллическими графами. Для расширения функциональности (например, учёта углов) можно модифицировать класс `ConvLayer`.

## Импортируем необходимые библиотеки

In [1]:
from torch_geometric.nn import global_mean_pool
import torch
import torch.nn as nn

In [97]:
# common libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import tqdm, os
import seaborn as sns
from datetime import date

from sklearn.metrics import r2_score, mean_absolute_error
from tqdm import tqdm

# torch libraries
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [99]:
# Torch geometric libraries
from torch_geometric.data import Data 
from torch_geometric.data import Dataset as Dataset_geometric
from torch_geometric.loader import DataLoader as DataLoader_geometric

# import modules
# from models.GNN_first import GCN

In [98]:
# assistive libraries
plt.rcParams.update(plt.rcParamsDefault) # Сброс настроек
import openpyxl # работа с excel

In [3]:
import time

In [4]:
import torch.optim as optim
from sklearn import metrics
from torch.autograd import Variable
from torch.optim.lr_scheduler import MultiStepLR

## Загрузка данных через parser

Разбираем часть кода связанного с парсером. Выяснили, что парсер нужен для запуска кода файла из командной строки. С помощью данного парсера из командной строки можно запускать различные функции и сообщать им необходимые параметры. В данной работе мы не используем парсер, но нужно разобраться как он работает.

В файле **[main.py](https://github.com/txie-93/cgcnn/blob/master/main.py)** CGCNN парсер (`argparse.ArgumentParser`) играет **ключевую роль в управлении конфигурацией** обучения и оценки модели. Вот детальный разбор его функций:

---

### **1. Основное назначение парсера**
Парсер обрабатывает **аргументы командной строки**, позволяя гибко настраивать:
- Пути к данным
- Гиперпараметры модели
- Параметры обучения
- Режимы работы (регрессия/классификация)

---

### **2. Ключевые группы аргументов**
#### **a) Данные (`data_options`, `--task`)**
```python
parser.add_argument('data_options', nargs='+', help='Path to dataset and options')
parser.add_argument('--task', choices=['regression', 'classification'], default='regression')
```
- **Роль**: Задает путь к данным (`root_dir`) и тип задачи (предсказание энергии/классификация).

#### **b) Параметры модели**
```python
parser.add_argument('--atom-fea-len', type=int, default=64, help='Размер атомных эмбеддингов')
parser.add_argument('--n-conv', type=int, default=3, help='Число графовых слоёв')
```
- **Роль**: Контролирует архитектуру CGCNN.

#### **c) Обучение**
```python
parser.add_argument('--epochs', type=int, default=30, help='Число эпох')
parser.add_argument('--lr', type=float, default=0.01, help='Learning rate')
```
- **Роль**: Управляет процессом обучения.

#### **d) Разделение данных**
```python
train_group = parser.add_mutually_exclusive_group()
train_group.add_argument('--train-ratio', type=float)
train_group.add_argument('--train-size', type=int)
```
- **Роль**: Позволяет задать размер обучающей выборки **либо** в процентах, **либо** в абсолютных числах.

---


### **3. Как парсер интегрирован в программу?**
#### **a) Инициализация**
```python
parser = argparse.ArgumentParser(description='Crystal Graph CNN')
```
Создает парсер с описанием программы.

#### **b) Парсинг аргументов**
```python
args = parser.parse_args()
```
Преобразует строку командной строки в объект `args` с атрибутами:
```python
args.epochs  # -> 30 (значение по умолчанию)
args.root_dir  # -> Путь из data_options
```

#### **c) Использование в коде**
Параметры применяются во всех критических компонентах:
```python
# Загрузка данных
dataset = CIFData(args.root_dir)

# Инициализация модели
model = CrystalGraphConvNet(
    atom_fea_len=args.atom_fea_len,
    n_conv=args.n_conv
)

# Обучение
train_loader = DataLoader(..., batch_size=args.batch_size)
```

---


### **4. Особенности реализации**
1. **Взаимоисключающие группы**:
   ```python
   train_group = parser.add_mutually_exclusive_group()
   ```
   Гарантирует, что можно указать **либо** `--train-ratio`, **либо** `--train-size`, но не оба сразу.

2. **Умолчания для CUDA**:
   ```python
   args.cuda = not args.disable_cuda and torch.cuda.is_available()
   ```
   Автоматически определяет использование GPU.

3. **Динамическая инициализация**:
   ```python
   if args.task == 'regression':
       criterion = nn.MSELoss()
   else:
       criterion = nn.CrossEntropyLoss()
   ```
   Выбирает функцию потерь в зависимости от задачи.

---



### **5. Пример запуска с аргументами**
```bash
python main.py ./data \
    --task regression \
    --epochs 100 \
    --batch-size 64 \
    --atom-fea-len 128
```
- `./data` → `args.data_options[0]`
- Остальные флаги → соответствующие атрибуты `args`.

---



### **6. Отличия от других реализаций**
| Особенность         | CGCNN (`main.py`)      | Стандартный подход      |
|---------------------|-----------------------|------------------------|
| **Управление данными** | Гибкие группы (`train-ratio`/`size`) | Фиксированные параметры |
| **Интеграция модели**  | Параметры архитектуры через CLI       | Жёстко закодированы     |
| **Режимы работы**    | Регрессия/классификация переключается флагом | Отдельные скрипты |

---

### **Вывод**
Парсер в CGCNN:
- **Стандартизирует** настройку экспериментов через командную строку
- **Упрощает** воспроизводимость (все параметры сохраняются в командной строке)
- **Делает код модульным** — можно менять гиперпараметры без правки исходников

Это ключевой компонент, превращающий `main.py` в **универсальный инструмент** для исследований в материаловедении.

In [5]:
# # Импорт модуля для обработки аргументов командной строки
# import argparse

In [6]:
# # Создание парсера аргументов с описанием программы
# parser = argparse.ArgumentParser(
#     description='Crystal Graph Convolutional Neural Networks'
# )
# print(parser)

Основные аргументы

In [7]:
# # Аргумент для опций датасета (обязательный, минимум 1 значение)
# parser.add_argument(
#     'data_options',  # Имя аргумента
#     metavar='OPTIONS',  # Имя для отображения в справке
#     nargs='+',  # Принимает один или несколько значений
#     help='dataset options, started with the path to root dir, then other options' # параметры набора данных, начиная с пути к корневому каталогу, затем другие параметры
# )

In [8]:
# parser.dest

In [9]:
# print(parser)

In [10]:
# parser.args()

In [11]:
# parser.parse_args()

In [12]:
# try:
#     args = parser.parse_args()
# except SystemExit:
#     # Подавляем ошибку для работы в Jupyter
#     args = parser.parse_args([])  # Пустой список = аргументы по умолчанию

In [13]:
# # Выбор типа задачи: регрессия или классификация (по умолчанию регрессия)
# parser.add_argument(
#     '--task',
#     choices=['regression', 'classification'],  # Допустимые значения
#     default='regression',  # Значение по умолчанию
#     help='complete a regression or classification task (default: regression)'
# )

In [14]:









# # Отключение CUDA (GPU)
# parser.add_argument(
#     '--disable-cuda',
#     action='store_true',  # Флаг (True если указан)
#     help='Disable CUDA'
# )

# # Параметры данных
# # ---------------------------------------------------------------------
# # Количество workers для загрузки данных (по умолчанию 0 - основной поток)
# parser.add_argument(
#     '-j', '--workers',  # Короткое и полное имя
#     default=0,  # Значение по умолчанию
#     type=int,  # Тип значения
#     metavar='N',  # Имя для отображения
#     help='number of data loading workers (default: 0)'
# )

# # Параметры обучения
# # ---------------------------------------------------------------------
# # Количество эпох обучения
# parser.add_argument(
#     '--epochs',
#     default=30,
#     type=int,
#     metavar='N',
#     help='number of total epochs to run (default: 30)'
# )

# # Стартовая эпоха (полезно при возобновлении обучения)
# parser.add_argument(
#     '--start-epoch',
#     default=0,
#     type=int,
#     metavar='N',
#     help='manual epoch number (useful on restarts)'
# )

# # Размер батча
# parser.add_argument(
#     '-b', '--batch-size',
#     default=256,
#     type=int,
#     metavar='N',
#     help='mini-batch size (default: 256)'
# )

# # Скорость обучения
# parser.add_argument(
#     '--lr', '--learning-rate',
#     default=0.01,
#     type=float,
#     metavar='LR',
#     help='initial learning rate (default: 0.01)'
# )

# # Эпохи для изменения lr (может быть несколько значений)
# parser.add_argument(
#     '--lr-milestones',
#     default=[100],
#     nargs='+',  # Один или несколько значений
#     type=int,
#     metavar='N',
#     help='milestones for scheduler (default: [100])'
# )

# # Момент для оптимизатора
# parser.add_argument(
#     '--momentum',
#     default=0.9,
#     type=float,
#     metavar='M',
#     help='momentum'
# )

# # Вес для L2-регуляризации
# parser.add_argument(
#     '--weight-decay', '--wd',
#     default=0,
#     type=float,
#     metavar='W',
#     help='weight decay (default: 0)'
# )

# # Частота вывода информации
# parser.add_argument(
#     '--print-freq', '-p',
#     default=10,
#     type=int,
#     metavar='N',
#     help='print frequency (default: 10)'
# )

# # Путь для возобновления обучения
# parser.add_argument(
#     '--resume',
#     default='',
#     type=str,
#     metavar='PATH',
#     help='path to latest checkpoint (default: none)'
# )

# # Группы для разделения данных
# # ---------------------------------------------------------------------
# # Взаимоисключающая группа для тренировочных данных
# train_group = parser.add_mutually_exclusive_group()
# train_group.add_argument(
#     '--train-ratio',
#     default=None,
#     type=float,
#     metavar='N',
#     help='number of training data to be loaded (default none)'
# )
# train_group.add_argument(
#     '--train-size',
#     default=None,
#     type=int,
#     metavar='N',
#     help='number of training data to be loaded (default none)'
# )

# # Взаимоисключающая группа для валидационных данных
# valid_group = parser.add_mutually_exclusive_group()
# valid_group.add_argument(
#     '--val-ratio',
#     default=0.1,
#     type=float,
#     metavar='N',
#     help='percentage of validation data to be loaded (default 0.1)'
# )
# valid_group.add_argument(
#     '--val-size',
#     default=None,
#     type=int,
#     metavar='N',
#     help='number of validation data to be loaded (default 1000)'
# )

# # Взаимоисключающая группа для тестовых данных
# test_group = parser.add_mutually_exclusive_group()
# test_group.add_argument(
#     '--test-ratio',
#     default=0.1,
#     type=float,
#     metavar='N',
#     help='percentage of test data to be loaded (default 0.1)'
# )
# test_group.add_argument(
#     '--test-size',
#     default=None,
#     type=int,
#     metavar='N',
#     help='number of test data to be loaded (default 1000)'
# )

# # Параметры модели
# # ---------------------------------------------------------------------
# # Выбор оптимизатора
# parser.add_argument(
#     '--optim',
#     default='SGD',
#     type=str,
#     metavar='SGD',
#     help='choose an optimizer, SGD or Adam, (default: SGD)'
# )

# # Размерность атомных признаков
# parser.add_argument(
#     '--atom-fea-len',
#     default=64,
#     type=int,
#     metavar='N',
#     help='number of hidden atom features in conv layers'
# )

# # Размерность скрытого слоя после пулинга
# parser.add_argument(
#     '--h-fea-len',
#     default=128,
#     type=int,
#     metavar='N',
#     help='number of hidden features after pooling'
# )

# # Количество сверточных слоев
# parser.add_argument(
#     '--n-conv',
#     default=3,
#     type=int,
#     metavar='N',
#     help='number of conv layers'
# )

# # Количество скрытых слоев после пулинга
# parser.add_argument(
#     '--n-h',
#     default=1,
#     type=int,
#     metavar='N',
#     help='number of hidden layers after pooling'
# )

# # Разбор аргументов командной строки
# args = parser.parse_args(sys.argv[1:])

# # Определение доступности CUDA
# args.cuda = not args.disable_cuda and torch.cuda.is_available()

# # Инициализация переменной для лучшего результата
# if args.task == 'regression':
#     best_mae_error = 1e10  # Для регрессии - минимизация ошибки
# else:
#     best_mae_error = 0  # Для классификации - максимизация точности

In [15]:
# import argparse

In [16]:
# parser = argparse.ArgumentParser(description='Crystal Graph Convolutional Neural Networks')
# print(parser)

In [17]:
# parser

In [18]:

# parser.add_argument('data_options', metavar='OPTIONS', nargs='+',
#                     help='dataset options, started with the path to root dir, '
#                          'then other options')
# parser.add_argument('--task', choices=['regression', 'classification'],
#                     default='regression', help='complete a regression or '
#                                                    'classification task (default: regression)')
# parser.add_argument('--disable-cuda', action='store_true',
#                     help='Disable CUDA')
# parser.add_argument('-j', '--workers', default=0, type=int, metavar='N',
#                     help='number of data loading workers (default: 0)')
# parser.add_argument('--epochs', default=30, type=int, metavar='N',
#                     help='number of total epochs to run (default: 30)')
# parser.add_argument('--start-epoch', default=0, type=int, metavar='N',
#                     help='manual epoch number (useful on restarts)')
# parser.add_argument('-b', '--batch-size', default=256, type=int,
#                     metavar='N', help='mini-batch size (default: 256)')
# parser.add_argument('--lr', '--learning-rate', default=0.01, type=float,
#                     metavar='LR', help='initial learning rate (default: '
#                                        '0.01)')
# parser.add_argument('--lr-milestones', default=[100], nargs='+', type=int,
#                     metavar='N', help='milestones for scheduler (default: '
#                                       '[100])')
# parser.add_argument('--momentum', default=0.9, type=float, metavar='M',
#                     help='momentum')
# parser.add_argument('--weight-decay', '--wd', default=0, type=float,
#                     metavar='W', help='weight decay (default: 0)')
# parser.add_argument('--print-freq', '-p', default=10, type=int,
#                     metavar='N', help='print frequency (default: 10)')
# parser.add_argument('--resume', default='', type=str, metavar='PATH',
#                     help='path to latest checkpoint (default: none)')
# train_group = parser.add_mutually_exclusive_group()
# train_group.add_argument('--train-ratio', default=None, type=float, metavar='N',
#                     help='number of training data to be loaded (default none)')
# train_group.add_argument('--train-size', default=None, type=int, metavar='N',
#                          help='number of training data to be loaded (default none)')
# valid_group = parser.add_mutually_exclusive_group()
# valid_group.add_argument('--val-ratio', default=0.1, type=float, metavar='N',
#                     help='percentage of validation data to be loaded (default '
#                          '0.1)')
# valid_group.add_argument('--val-size', default=None, type=int, metavar='N',
#                          help='number of validation data to be loaded (default '
#                               '1000)')
# test_group = parser.add_mutually_exclusive_group()
# test_group.add_argument('--test-ratio', default=0.1, type=float, metavar='N',
#                     help='percentage of test data to be loaded (default 0.1)')
# test_group.add_argument('--test-size', default=None, type=int, metavar='N',
#                         help='number of test data to be loaded (default 1000)')

# parser.add_argument('--optim', default='SGD', type=str, metavar='SGD',
#                     help='choose an optimizer, SGD or Adam, (default: SGD)')
# parser.add_argument('--atom-fea-len', default=64, type=int, metavar='N',
#                     help='number of hidden atom features in conv layers')
# parser.add_argument('--h-fea-len', default=128, type=int, metavar='N',
#                     help='number of hidden features after pooling')
# parser.add_argument('--n-conv', default=3, type=int, metavar='N',
#                     help='number of conv layers')
# parser.add_argument('--n-h', default=1, type=int, metavar='N',
#                     help='number of hidden layers after pooling')

# args = parser.parse_args(sys.argv[1:])

# args.cuda = not args.disable_cuda and torch.cuda.is_available()

# if args.task == 'regression':
#     best_mae_error = 1e10
# else:
#     best_mae_error = 0.

Верно! Если вы работаете с моделью **CGCNN напрямую в Jupyter Notebook**, парсер аргументов командной строки (`argparse`) вам **не нужен**. Вот как адаптировать код для ноутбука:

---

### **1. Замена парсера на ручные параметры**
Вместо:
```python
parser = argparse.ArgumentParser()
parser.add_argument('--epochs', default=30, type=int)
args = parser.parse_args()
```
Просто задайте параметры как переменные:
```python
epochs = 30         # Вместо args.epochs
batch_size = 256    # Вместо args.batch_size
task = "regression" # Вместо args.task
```

---

### **2. Пример настройки модели в Jupyter**
```python
# Параметры данных
root_dir = "./data"  # Путь к CIF-файлам
train_ratio = 0.8    # Доля обучающей выборки

# Параметры модели
atom_fea_len = 64    # Размер атомных эмбеддингов
n_conv = 3           # Число графовых слоёв

# Параметры обучения
lr = 0.01            # Скорость обучения
epochs = 50          # Число эпох

# Инициализация модели
model = CrystalGraphConvNet(
    orig_atom_fea_len=92,  # Для атомных номеров
    nbr_fea_len=5,         # Для межатомных расстояний
    atom_fea_len=atom_fea_len,
    n_conv=n_conv,
    h_fea_len=128
)

# Пример цикла обучения
for epoch in range(epochs):
    train(model, lr, batch_size)
```

---

### **3. Что делать с обязательным `data_options`?**
В оригинальном коде `data_options` — позиционный аргумент. В Jupyter замените его на переменную:
```python
# Вместо: 
# python main.py ./data --epochs 50
# В Jupyter:
data_path = "./data"  # Аналог data_options[0]
```

---

### **4. Как упростить загрузку данных**
Оригинальный `CIFData` требует `root_dir` из аргументов. Адаптируем:
```python
from cgcnn.data import CIFData

dataset = CIFData(root_dir="./data")  # Прямое указание пути
train_loader = DataLoader(dataset, batch_size=256, shuffle=True)
```

---

### **5. Плюсы отказа от парсера в Jupyter**
- **Наглядность**: Все параметры видны в одном месте.
- **Гибкость**: Меняйте значения без перезапуска терминала.
- **Интерактивность**: Тестируйте разные параметры в реальном времени.

---

### **6. Когда парсер всё же полезен в Jupyter**
Если хотите **сохранить совместимость** с оригинальным кодом, эмулируйте аргументы:
```python
import sys
sys.argv = ["", "./data", "--epochs", "50"]  # Эмуляция командной строки
args = parser.parse_args()
```

---

### **7. Пример end-to-end workflow**
```python
# 1. Импорт
from cgcnn.model import CrystalGraphConvNet
from cgcnn.data import CIFData
from torch.utils.data import DataLoader

# 2. Ручные параметры
params = {
    "data_path": "./data",
    "atom_fea_len": 64,
    "epochs": 30,
    "batch_size": 128
}

# 3. Загрузка данных
dataset = CIFData(root_dir=params["data_path"])
loader = DataLoader(dataset, batch_size=params["batch_size"])

# 4. Обучение
model = CrystalGraphConvNet(...)
for epoch in range(params["epochs"]):
    train(...)
```

---

### **Вывод**
Для Jupyter Notebook:
1. **Замените `argparse` на обычные переменные**.
2. **Прямо передавайте параметры** в классы модели и данных.
3. **Совет**: Выносите параметры в отдельный словарь (как в примере выше) для удобства управления.

Парсер нужен только для запуска через командную строку — в интерактивной среде он избыточен.

## Модель CGCNN

In [19]:
class ConvLayer(nn.Module):
    """
    Convolutional operation on graphs
    Определяет слой сверточной нейронной сети для обработки атомных признаков и их соседей.
    """
    def __init__(self, atom_fea_len, nbr_fea_len):
        """
        Initialize ConvLayer.

        Parameters:
        ----------
        atom_fea_len (int): Number of atom hidden features. Размерность вектора признаков атома
        nbr_fea_len (int): Number of bond features. Размерность вектора признаков соседей
        """
        super(ConvLayer, self).__init__()

        self.atom_fea_len = atom_fea_len # Размерность вектора признаков атома
        self.nbr_fea_len = nbr_fea_len # Размерность вектора признаков соседей
        self.fc_full = nn.Linear(2*self.atom_fea_len+self.nbr_fea_len,
                                 2*self.atom_fea_len) # Полносвязный слой для объединения признаков атома и его соседей
        
        self.sigmoid = nn.Sigmoid() # Функция активации сигмоид для фильтрации признаков       
        self.softplus1 = nn.Softplus() # Функция активации softplus для обработки признаков

        self.bn1 = nn.BatchNorm1d(2*self.atom_fea_len) # Нормализация батча для стабилизации обучения
        self.bn2 = nn.BatchNorm1d(self.atom_fea_len) # Нормализация батча для выходных признаков
        self.softplus2 = nn.Softplus()

    def forward(self, atom_in_fea, nbr_fea, nbr_fea_idx):
        """
        Forward pass  Прямой проход данных через слой.

        N: Total number of atoms in the batch
        M: Max number of neighbors

        Parameters
        ----------

        atom_in_fea: Variable(torch.Tensor) shape (N, atom_fea_len)
          Atom hidden features before convolution
        nbr_fea: Variable(torch.Tensor) shape (N, M, nbr_fea_len)
          Bond features of each atom's M neighbors
        nbr_fea_idx: torch.LongTensor shape (N, M)
          Indices of M neighbors of each atom

        Returns
        -------

        atom_out_fea: nn.Variable shape (N, atom_fea_len)
          Atom hidden features after convolution

        """
        # TODO will there be problems with the index zero padding?
        N, M = nbr_fea_idx.shape  # N - число атомов, M - число соседей для каждого атома
        
        # convolution
        atom_nbr_fea = atom_in_fea[nbr_fea_idx, :] # Извлечение признаков соседних атомов по индексам
        
        # Объединение признаков центрального атома, его соседей и их взаимодействий
        total_nbr_fea = torch.cat(
            [atom_in_fea.unsqueeze(1).expand(N, M, self.atom_fea_len),
             atom_nbr_fea, nbr_fea], dim=2) 
        total_gated_fea = self.fc_full(total_nbr_fea) # Пропуск через полносвязный слой
        
        # Применение нормализации и активации
        total_gated_fea = self.bn1(total_gated_fea.view(
            -1, self.atom_fea_len*2)).view(N, M, self.atom_fea_len*2)
        
        nbr_filter, nbr_core = total_gated_fea.chunk(2, dim=2) # Разделение на две части
        nbr_filter = self.sigmoid(nbr_filter) # Фильтр признаков
        nbr_core = self.softplus1(nbr_core) # Основные признаки
        nbr_sumed = torch.sum(nbr_filter * nbr_core, dim=1)
        nbr_sumed = self.bn2(nbr_sumed)
        out = self.softplus2(atom_in_fea + nbr_sumed) # Финальная активация и обновление признаков атома
        return out

In [20]:
class CrystalGraphConvNet(nn.Module):
    """
    Create a crystal graph convolutional neural network for predicting total
    material properties.
    Определяет всю архитектуру сверточной нейронной сети на графах для предсказания свойств кристаллов.
    """
    def __init__(self, orig_atom_fea_len, nbr_fea_len,
                 atom_fea_len=64, n_conv=3, h_fea_len=128, n_h=1,
                 classification=False):
        """
        Initialize CrystalGraphConvNet. Инициализация сети.

        Parameters
        ----------
        orig_atom_fea_len: int
          Number of atom features in the input. Исходная размерность вектора признаков атома.
        nbr_fea_len: int
          Number of bond features. Количество элементов связи.
        atom_fea_len: int
          Number of hidden atom features in the convolutional layers. Количество скрытых элементов атома в сверточных слоях
        n_conv: int
          Number of convolutional layers
        h_fea_len: int
          Number of hidden features after pooling
        n_h: int
          Number of hidden layers after pooling
        """
        super(CrystalGraphConvNet, self).__init__()
        self.classification = classification
        
        # Линейный слой для преобразования исходных признаков атома в скрытое пространство
        self.embedding = nn.Linear(orig_atom_fea_len, atom_fea_len)
        # Создание списка сверточных слоев
        self.convs = nn.ModuleList([ConvLayer(atom_fea_len=atom_fea_len,
                                    nbr_fea_len=nbr_fea_len)
                                    for _ in range(n_conv)])
        # Линейный слой для преобразования после свертки перед полносвязными слоями
        self.conv_to_fc = nn.Linear(atom_fea_len, h_fea_len)
        self.conv_to_fc_softplus = nn.Softplus()
        
        # Создание полносвязных слоев и функций активации для них
        if n_h > 1:
            self.fcs = nn.ModuleList([nn.Linear(h_fea_len, h_fea_len)
                                      for _ in range(n_h-1)])
            self.softpluses = nn.ModuleList([nn.Softplus()
                                             for _ in range(n_h-1)])
        # Выходной слой: для классификации - 2 нейрона, для регрессии - 1 нейрон
        if self.classification:
            self.fc_out = nn.Linear(h_fea_len, 2)
        else:
            self.fc_out = nn.Linear(h_fea_len, 1)
        
        # Если задача классификации, добавляем слой Dropout для регуляризации
        if self.classification:
            self.logsoftmax = nn.LogSoftmax(dim=1)
            self.dropout = nn.Dropout()

    def forward(self, atom_fea, nbr_fea, nbr_fea_idx, crystal_atom_idx):
        """
        Forward pass  Прямой проход данных через модель.

        N: Total number of atoms in the batch
        M: Max number of neighbors
        N0: Total number of crystals in the batch

        Parameters
        ----------

        atom_fea: Variable(torch.Tensor) shape (N, orig_atom_fea_len)
          Atom features from atom type
        nbr_fea: Variable(torch.Tensor) shape (N, M, nbr_fea_len)
          Bond features of each atom's M neighbors
        nbr_fea_idx: torch.LongTensor shape (N, M)
          Indices of M neighbors of each atom
        crystal_atom_idx: list of torch.LongTensor of length N0
          Mapping from the crystal idx to atom idx   Отображение из crystal idx в atom idx

        Returns
        -------

        prediction: nn.Variable shape (N, )
          Atom hidden features after convolution

        """
        # Преобразование исходных признаков атомов в эмбеддинги
        atom_fea = self.embedding(atom_fea)

        # Последовательное применение сверточных слоев
        for conv_func in self.convs:
            atom_fea = conv_func(atom_fea, nbr_fea, nbr_fea_idx)
        # Агрегация признаков атомов для каждого кристалла
        crys_fea = self.pooling(atom_fea, crystal_atom_idx)
        crys_fea = self.conv_to_fc(self.conv_to_fc_softplus(crys_fea))
        
        # Преобразование после свертки перед полносвязными слоями
        crys_fea = self.conv_to_fc_softplus(crys_fea)
        
        # Если задача классификации, применяем Dropout перед выходным слоем
        if self.classification:
            crys_fea = self.dropout(crys_fea)
        
        # Последовательное применение полносвязных слоев с функцией активации Softplus
        if hasattr(self, 'fcs') and hasattr(self, 'softpluses'):
            for fc, softplus in zip(self.fcs, self.softpluses):
                crys_fea = softplus(fc(crys_fea))
        
        # Выходной слой для получения предсказаний
        out = self.fc_out(crys_fea)
        
        # Если задача классификации, применяем логарифмическую софтмакс-функцию для получения лог-вероятностей классов
        if self.classification:
            out = self.logsoftmax(out)
        return out

    def pooling(self, atom_fea, crystal_atom_idx):
        """
        Pooling the atom features to crystal features / Агрегация (пулинг) признаков атомов для каждого кристалла.

        N: Total number of atoms in the batch 
        N0: Total number of crystals in the batch 

        Parameters
        ----------

        atom_fea: Variable(torch.Tensor) shape (N, atom_fea_len)
          Atom feature vectors of the batch
        crystal_atom_idx: list of torch.LongTensor of length N0
          Mapping from the crystal idx to atom idx
        """
        assert sum([len(idx_map) for idx_map in crystal_atom_idx]) ==\
            atom_fea.data.shape[0]
        
        # Инициализация списка для хранения агрегированных признаков каждого кристалла
        summed_fea = [torch.mean(atom_fea[idx_map], dim=0, keepdim=True)
                      for idx_map in crystal_atom_idx]
        # Объединение списка в тензор
        return torch.cat(summed_fea, dim=0)

## Вспомогательные функции

In [ ]:
# def train(train_loader, model, criterion, optimizer, epoch, normalizer):
#     batch_time = AverageMeter()
#     data_time = AverageMeter()
#     losses = AverageMeter()
#     if args.task == 'regression':
#         mae_errors = AverageMeter()
#     else:
#         accuracies = AverageMeter()
#         precisions = AverageMeter()
#         recalls = AverageMeter()
#         fscores = AverageMeter()
#         auc_scores = AverageMeter()

#     # switch to train mode
#     model.train()

#     end = time.time()
#     for i, (input, target, _) in enumerate(train_loader):
#         # measure data loading time
#         data_time.update(time.time() - end)

#         if args.cuda:
#             input_var = (Variable(input[0].cuda(non_blocking=True)),
#                          Variable(input[1].cuda(non_blocking=True)),
#                          input[2].cuda(non_blocking=True),
#                          [crys_idx.cuda(non_blocking=True) for crys_idx in input[3]])
#         else:
#             input_var = (Variable(input[0]),
#                          Variable(input[1]),
#                          input[2],
#                          input[3])
#         # normalize target
#         if args.task == 'regression':
#             target_normed = normalizer.norm(target)
#         else:
#             target_normed = target.view(-1).long()
#         if args.cuda:
#             target_var = Variable(target_normed.cuda(non_blocking=True))
#         else:
#             target_var = Variable(target_normed)

#         # compute output
#         output = model(*input_var)
#         loss = criterion(output, target_var)

#         # measure accuracy and record loss
#         if args.task == 'regression':
#             mae_error = mae(normalizer.denorm(output.data.cpu()), target)
#             losses.update(loss.data.cpu(), target.size(0))
#             mae_errors.update(mae_error, target.size(0))
#         else:
#             accuracy, precision, recall, fscore, auc_score = \
#                 class_eval(output.data.cpu(), target)
#             losses.update(loss.data.cpu().item(), target.size(0))
#             accuracies.update(accuracy, target.size(0))
#             precisions.update(precision, target.size(0))
#             recalls.update(recall, target.size(0))
#             fscores.update(fscore, target.size(0))
#             auc_scores.update(auc_score, target.size(0))

#         # compute gradient and do SGD step
#         optimizer.zero_grad()
#         loss.backward()
#         optimizer.step()

#         # measure elapsed time
#         batch_time.update(time.time() - end)
#         end = time.time()

#         if i % args.print_freq == 0:
#             if args.task == 'regression':
#                 print('Epoch: [{0}][{1}/{2}]\t'
#                       'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                       'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
#                       'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                       'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
#                     epoch, i, len(train_loader), batch_time=batch_time,
#                     data_time=data_time, loss=losses, mae_errors=mae_errors)
#                 )
#             else:
#                 print('Epoch: [{0}][{1}/{2}]\t'
#                       'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
#                       'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
#                       'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
#                       'Accu {accu.val:.3f} ({accu.avg:.3f})\t'
#                       'Precision {prec.val:.3f} ({prec.avg:.3f})\t'
#                       'Recall {recall.val:.3f} ({recall.avg:.3f})\t'
#                       'F1 {f1.val:.3f} ({f1.avg:.3f})\t'
#                       'AUC {auc.val:.3f} ({auc.avg:.3f})'.format(
#                     epoch, i, len(train_loader), batch_time=batch_time,
#                     data_time=data_time, loss=losses, accu=accuracies,
#                     prec=precisions, recall=recalls, f1=fscores,
#                     auc=auc_scores)
#                 )

### class Normalizer

In [ ]:
# class Normalizer(object):
#     """Normalize a Tensor and restore it later. """

#     def __init__(self, tensor):
#         """tensor is taken as a sample to calculate the mean and std"""
#         self.mean = torch.mean(tensor)
#         self.std = torch.std(tensor)

#     def norm(self, tensor):
#         return (tensor - self.mean) / self.std

#     def denorm(self, normed_tensor):
#         return normed_tensor * self.std + self.mean

#     def state_dict(self):
#         return {'mean': self.mean,
#                 'std': self.std}

#     def load_state_dict(self, state_dict):
#         self.mean = state_dict['mean']
#         self.std = state_dict['std']


# def mae(prediction, target):
#     """
#     Computes the mean absolute error between prediction and target

#     Parameters
#     ----------

#     prediction: torch.Tensor (N, 1)
#     target: torch.Tensor (N, 1)
#     """
#     return torch.mean(torch.abs(target - prediction))


# def class_eval(prediction, target):
#     prediction = np.exp(prediction.numpy())
#     target = target.numpy()
#     pred_label = np.argmax(prediction, axis=1)
#     target_label = np.squeeze(target)
#     if not target_label.shape:
#         target_label = np.asarray([target_label])
#     if prediction.shape[1] == 2:
#         precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
#             target_label, pred_label, average='binary')
#         auc_score = metrics.roc_auc_score(target_label, prediction[:, 1])
#         accuracy = metrics.accuracy_score(target_label, pred_label)
#     else:
#         raise NotImplementedError
#     return accuracy, precision, recall, fscore, auc_score

In [100]:
class Normalizer(object):
    """Нормализатор тензоров: выполняет нормализацию и денормализацию данных.
    
    Используется для приведения данных к нулевому среднему и единичной дисперсии.
    """

    def __init__(self, tensor):
        """Инициализация на основе примера данных.
        
        Args:
            tensor (torch.Tensor): Тензор-пример для вычисления среднего и стандартного отклонения.
        """
        # Вычисляем среднее значение и стандартное отклонение по входному тензору
        self.mean = torch.mean(tensor)
        self.std = torch.std(tensor)

    def norm(self, tensor):
        """Нормализация входного тензора.
        
        Args:
            tensor (torch.Tensor): Входной тензор для нормализации.
        Returns:
            torch.Tensor: Нормализованный тензор.
        """
        return (tensor - self.mean) / self.std

    def denorm(self, normed_tensor):
        """Обратное преобразование нормализованного тензора.
        
        Args:
            normed_tensor (torch.Tensor): Нормализованный тензор.
        Returns:
            torch.Tensor: Тензор в исходной шкале.
        """
        return normed_tensor * self.std + self.mean

    def state_dict(self):
        """Возвращает состояние нормализатора для сохранения.
        
        Returns:
            dict: Словарь с параметрами нормализатора.
        """
        return {'mean': self.mean,
                'std': self.std}

    def load_state_dict(self, state_dict):
        """Загружает состояние нормализатора.
        
        Args:
            state_dict (dict): Словарь с параметрами нормализатора.
        """
        self.mean = state_dict['mean']
        self.std = state_dict['std']


def mae(prediction, target):
    """Вычисляет среднюю абсолютную ошибку (Mean Absolute Error) между предсказанием и целью.
    
    Args:
        prediction (torch.Tensor): Тензор предсказаний размерности (N, 1).
        target (torch.Tensor): Тензор истинных значений размерности (N, 1).
    
    Returns:
        torch.Tensor: Значение MAE.
    """
    return torch.mean(torch.abs(target - prediction))


def class_eval(prediction, target):
    """Вычисляет метрики качества для задачи классификации.
    
    Args:
        prediction (torch.Tensor): Выход модели до softmax.
        target (torch.Tensor): Истинные метки классов.
    
    Returns:
        tuple: (accuracy, precision, recall, fscore, auc_score)
    """
    # Применяем exp для получения ненормализованных вероятностей (аналог softmax)
    prediction = np.exp(prediction.numpy())
    target = target.numpy()
    
    # Получаем предсказанные метки (класс с максимальной вероятностью)
    pred_label = np.argmax(prediction, axis=1)
    
    # Обеспечиваем корректную форму для целевых меток
    target_label = np.squeeze(target)
    if not target_label.shape:
        target_label = np.asarray([target_label])
    
    # Вычисляем метрики только для бинарной классификации
    if prediction.shape[1] == 2:
        # precision (точность), recall (полнота), fscore (F1-мера)
        precision, recall, fscore, _ = metrics.precision_recall_fscore_support(
            target_label, pred_label, average='binary')
        
        # AUC-ROC (площадь под ROC-кривой)
        auc_score = metrics.roc_auc_score(target_label, prediction[:, 1])
        
        # accuracy (доля правильных ответов)
        accuracy = metrics.accuracy_score(target_label, pred_label)
    else:
        raise NotImplementedError("Поддерживается только бинарная классификация")
    
    return accuracy, precision, recall, fscore, auc_score

Ключевые особенности:

1. **Normalizer**:
   - Работает по принципу стандартизации: `(x - mean) / std`
   - Сохраняет состояние (mean и std) для согласованной нормализации/денормализации
   - Позволяет сохранять и загружать параметры нормализации

2. **mae()**:
   - Простая реализация средней абсолютной ошибки
   - Устойчива к выбросам по сравнению с MSE

3. **class_eval()**:
   - Рассчитывает комплекс метрик для бинарной классификации
   - Использует numpy для совместимости с sklearn.metrics
   - Включает как threshold-метрики (accuracy, precision, recall), так и score-метрику (AUC-ROC)
   - Автоматически обрабатывает разные форматы входных меток

Важные нюансы:
- Normalizer вычисляет статистики по всему тензору (а не по измерениям)
- class_eval ожидает на входе логарифмы вероятностей (логиты)
- Для мультиклассовой классификации потребуется модификация class_eval

### class AverageMeter

In [23]:
class AverageMeter(object):
    """Computes and stores the average and current value"""

    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

### def save_checkpoint ????

In [24]:
import shutil

In [25]:
def save_checkpoint(state, is_best, filename='checkpoint.pth.tar'):
    torch.save(state, filename)
    if is_best:
        shutil.copyfile(filename, 'model_best.pth.tar')

### def adjust_learning_rate  ?????

In [26]:
def adjust_learning_rate(optimizer, epoch, k):
    """Sets the learning rate to the initial LR decayed by 10 every k epochs"""
    assert type(k) is int
    lr = args.lr * (0.1 ** (epoch // k))
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

### def validate ??????

In [ ]:
# зададим глобальные переменные
args_task_ = 'regression'
args_cuda_ = None
args_print_freq_ = 4

In [ ]:
def validate(val_loader, model, criterion, normalizer, test=False):
    """
    Функция для валидации/тестирования модели
    
    Параметры:
        val_loader (DataLoader): Загрузчик данных для валидации/теста
        model (nn.Module): Модель для валидации
        criterion: Функция потерь
        normalizer: Нормализатор для целевых значений
        test (bool): Флаг режима тестирования (если True - сохраняет предсказания)
    """
    
    # Инициализация счетчиков метрик
    batch_time = AverageMeter()  # Время обработки батча
    losses = AverageMeter()      # Значения функции потерь
    
    # Инициализация метрик в зависимости от типа задачи
    if args_task_ == 'regression':
        mae_errors = AverageMeter()  # Средняя абсолютная ошибка
    else:
        # Метрики классификации
        accuracies = AverageMeter()  # Точность
        precisions = AverageMeter()  # Прецизионность
        recalls = AverageMeter()     # Полнота
        fscores = AverageMeter()     # F1-мера
        auc_scores = AverageMeter()  # AUC-ROC
    
    # Инициализация списков для сохранения результатов (в режиме теста)
    if test:
        test_targets = []    # Истинные значения
        test_preds = []      # Предсказания модели
        test_cif_ids = []    # Идентификаторы образцов

    # Переводим модель в режим оценки (отключаем dropout/batchnorm)
    model.eval()

    end = time.time()  # Засекаем время
    
    # Основной цикл по данным
    for i, (input, target, batch_cif_ids) in enumerate(val_loader):
        # Подготовка входных данных
        if args_cuda_:
            with torch.no_grad():  # Отключаем вычисление градиентов
                # Переносим данные на GPU
                input_var = (
                    Variable(input[0].cuda(non_blocking=True)),  # Основные признаки
                    Variable(input[1].cuda(non_blocking=True)),  # Признаки соседей
                    input[2].cuda(non_blocking=True),           # Индексы
                    [crys_idx.cuda(non_blocking=True) for crys_idx in input[3]]  # Кристаллические индексы
                )
        else:
            with torch.no_grad():
                # CPU-версия
                input_var = (
                    Variable(input[0]),
                    Variable(input[1]),
                    input[2],
                    input[3]
                )

        # Нормализация целевых значений
        if args_task_ == 'regression':
            target_normed = normalizer.norm(target)  # Нормализуем для регрессии
        else:
            target_normed = target.view(-1).long()  # Преобразуем для классификации
        
        # Подготовка целевых переменных
        if args_cuda_:
            with torch.no_grad():
                target_var = Variable(target_normed.cuda(non_blocking=True))
        else:
            with torch.no_grad():
                target_var = Variable(target_normed)

        # Прямой проход (forward pass)
        output = model(*input_var)        # Получаем предсказания
        loss = criterion(output, target_var)  # Вычисляем потери

        # Вычисление и обновление метрик
        if args_task_ == 'regression':
            # Денормализуем предсказания для расчета MAE
            mae_error = mae(normalizer.denorm(output.data.cpu()), target)
            losses.update(loss.data.cpu().item(), target.size(0))
            mae_errors.update(mae_error, target.size(0))
            
            if test:
                # Сохраняем предсказания для регрессии
                test_pred = normalizer.denorm(output.data.cpu())
                test_target = target
                test_preds += test_pred.view(-1).tolist()
                test_targets += test_target.view(-1).tolist()
                test_cif_ids += batch_cif_ids
        else:
            # Вычисляем метрики классификации
            accuracy, precision, recall, fscore, auc_score = \
                class_eval(output.data.cpu(), target)
            losses.update(loss.data.cpu().item(), target.size(0))
            accuracies.update(accuracy, target.size(0))
            precisions.update(precision, target.size(0))
            recalls.update(recall, target.size(0))
            fscores.update(fscore, target.size(0))
            auc_scores.update(auc_score, target.size(0))
            
            if test:
                # Сохраняем предсказания для классификации
                test_pred = torch.exp(output.data.cpu())  # Получаем вероятности
                test_target = target
                assert test_pred.shape[1] == 2  # Проверка для бинарной классификации
                test_preds += test_pred[:, 1].tolist()  # Вероятность положительного класса
                test_targets += test_target.view(-1).tolist()
                test_cif_ids += batch_cif_ids

        # Обновление времени обработки
        batch_time.update(time.time() - end)
        end = time.time()

        # Периодический вывод статистики
        if i % args_print_freq_ == 0:
            if args_task_ == 'regression':
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
                    i, len(val_loader), batch_time=batch_time, loss=losses,
                    mae_errors=mae_errors))
            else:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Accu {accu.val:.3f} ({accu.avg:.3f})\t'
                      'Precision {prec.val:.3f} ({prec.avg:.3f})\t'
                      'Recall {recall.val:.3f} ({recall.avg:.3f})\t'
                      'F1 {f1.val:.3f} ({f1.avg:.3f})\t'
                      'AUC {auc.val:.3f} ({auc.avg:.3f})'.format(
                    i, len(val_loader), batch_time=batch_time, loss=losses,
                    accu=accuracies, prec=precisions, recall=recalls,
                    f1=fscores, auc=auc_scores))

    # Сохранение результатов в режиме теста
    if test:
        star_label = '**'
        import csv
        with open('test_results.csv', 'w') as f:
            writer = csv.writer(f)
            for cif_id, target, pred in zip(test_cif_ids, test_targets, test_preds):
                writer.writerow((cif_id, target, pred))  # Запись в CSV: ID, истинное значение, предсказание
    else:
        star_label = '*'
    
    # Финальный вывод результатов
    if args_task_ == 'regression':
        print(' {star} MAE {mae_errors.avg:.3f}'.format(star=star_label,
                                                      mae_errors=mae_errors))
        return mae_errors.avg
    else:
        print(' {star} AUC {auc.avg:.3f}'.format(star=star_label,
                                               auc=auc_scores))
        return auc_scores.avg

In [363]:
def validate(val_loader, model, criterion, normalizer, test=False):
    batch_time = AverageMeter()
    losses = AverageMeter()
    if args_task_ == 'regression':
        mae_errors = AverageMeter()
    else:
        accuracies = AverageMeter()
        precisions = AverageMeter()
        recalls = AverageMeter()
        fscores = AverageMeter()
        auc_scores = AverageMeter()
    if test:
        test_targets = []
        test_preds = []
        test_cif_ids = []

    # switch to evaluate mode
    model.eval()

    end = time.time()
    for i, (input, target, batch_cif_ids) in enumerate(val_loader):
        if args_cuda_:
            with torch.no_grad():
                input_var = (Variable(input[0].cuda(non_blocking=True)),
                             Variable(input[1].cuda(non_blocking=True)),
                             input[2].cuda(non_blocking=True),
                             [crys_idx.cuda(non_blocking=True) for crys_idx in input[3]])
        else:
            with torch.no_grad():
                input_var = (Variable(input[0]),
                             Variable(input[1]),
                             input[2],
                             input[3])
        if args_task_ == 'regression':
            target_normed = normalizer.norm(target)
        else:
            target_normed = target.view(-1).long()
        if args_cuda_:
            with torch.no_grad():
                target_var = Variable(target_normed.cuda(non_blocking=True))
        else:
            with torch.no_grad():
                target_var = Variable(target_normed)

        # compute output
        output = model(*input_var)
        loss = criterion(output, target_var)

        # measure accuracy and record loss
        if args_task_ == 'regression':
            mae_error = mae(normalizer.denorm(output.data.cpu()), target)
            losses.update(loss.data.cpu().item(), target.size(0))
            mae_errors.update(mae_error, target.size(0))
            if test:
                test_pred = normalizer.denorm(output.data.cpu())
                test_target = target
                test_preds += test_pred.view(-1).tolist()
                test_targets += test_target.view(-1).tolist()
                test_cif_ids += batch_cif_ids
        else:
            accuracy, precision, recall, fscore, auc_score = \
                class_eval(output.data.cpu(), target)
            losses.update(loss.data.cpu().item(), target.size(0))
            accuracies.update(accuracy, target.size(0))
            precisions.update(precision, target.size(0))
            recalls.update(recall, target.size(0))
            fscores.update(fscore, target.size(0))
            auc_scores.update(auc_score, target.size(0))
            if test:
                test_pred = torch.exp(output.data.cpu())
                test_target = target
                assert test_pred.shape[1] == 2
                test_preds += test_pred[:, 1].tolist()
                test_targets += test_target.view(-1).tolist()
                test_cif_ids += batch_cif_ids

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if i % args_print_freq_ == 0:
            if args_task_ == 'regression':
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
                    i, len(val_loader), batch_time=batch_time, loss=losses,
                    mae_errors=mae_errors))
            else:
                print('Test: [{0}/{1}]\t'
                      'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                      'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                      'Accu {accu.val:.3f} ({accu.avg:.3f})\t'
                      'Precision {prec.val:.3f} ({prec.avg:.3f})\t'
                      'Recall {recall.val:.3f} ({recall.avg:.3f})\t'
                      'F1 {f1.val:.3f} ({f1.avg:.3f})\t'
                      'AUC {auc.val:.3f} ({auc.avg:.3f})'.format(
                    i, len(val_loader), batch_time=batch_time, loss=losses,
                    accu=accuracies, prec=precisions, recall=recalls,
                    f1=fscores, auc=auc_scores))

    if test:
        star_label = '**'
        import csv
        with open('test_results.csv', 'w') as f:
            writer = csv.writer(f)
            for cif_id, target, pred in zip(test_cif_ids, test_targets,
                                            test_preds):
                writer.writerow((cif_id, target, pred))
    else:
        star_label = '*'
    if args_task_ == 'regression':
        print(' {star} MAE {mae_errors.avg:.3f}'.format(star=star_label,
                                                        mae_errors=mae_errors))
        return mae_errors.avg
    else:
        print(' {star} AUC {auc.avg:.3f}'.format(star=star_label,
                                                 auc=auc_scores))
        return auc_scores.avg

### def train(...)

In [28]:
args_task_ = 'regression'
args_cuda_ = None

In [29]:
def train(train_loader, model, criterion, optimizer, epoch, normalizer):
    """
    Функция для обучения модели на одной эпохе
    Параметры:
        train_loader - DataLoader для обучающих данных
        model - модель нейронной сети
        criterion - функция потерь
        optimizer - оптимизатор
        epoch - номер текущей эпохи
        normalizer - нормализатор для целевых значений
    """
    
    # Инициализация счетчиков для мониторинга процесса обучения:
    
    # batch_time - время обработки одного батча
    batch_time = AverageMeter()
    print('batch_time', batch_time)
    # data_time - время загрузки данных
    data_time = AverageMeter()
    print('data_time', data_time)
    # losses - значения функции потерь
    losses = AverageMeter()
    print('losses', losses)
    
    # В зависимости от типа задачи инициализируем соответствующие метрики:
    if args_task_ == 'regression':
        # Для регрессии используем MAE (Mean Absolute Error)
        mae_errors = AverageMeter()
        print('mae_errors', mae_errors)
    else:
        # Для классификации используем несколько метрик:
        accuracies = AverageMeter()    # Точность
        precisions = AverageMeter()    # Прецизионность
        recalls = AverageMeter()       # Полнота
        fscores = AverageMeter()       # F1-мера
        auc_scores = AverageMeter()    # AUC-ROC

    # Переводим модель в режим обучения (это важно для слоев типа Dropout и BatchNorm)
    model.train()
    print('Сработало model.train()')

    # Фиксируем время начала обработки
    end = time.time()
    print('end', end)
    
    # Основной цикл обучения по батчам:
    print('Начинаем цикл обучения по батчам!')
    for i, (input, target, _) in tqdm(enumerate(train_loader)):
        print('Новый i-й шаг цикла:', i)
        print('input - входные данные модели', input)
        print('target - целевые значения', target)

        # input - входные данные модели
        # target - целевые значения
        # _ - дополнительные метаданные (не используются)
        
        # 1. Измеряем время загрузки данных
        data_time.update(time.time() - end)
        print('Измеряем время загрузки данных')

        # 2. Подготовка входных данных:
        # Переносим данные на GPU, если используется CUDA
        if args_cuda_:
            print('args_cuda_')
            input_var = (
                Variable(input[0].cuda(non_blocking=True)),  # Атомные признаки
                Variable(input[1].cuda(non_blocking=True)),  # Признаки соседей
                input[2].cuda(non_blocking=True),           # Индексы соседей
                [crys_idx.cuda(non_blocking=True) for crys_idx in input[3]]  # Индексы кристаллов
            )
        else:
            print('args_cuda_ else')
            input_var = (
                Variable(input[0]),
                Variable(input[1]),
                input[2],
                input[3]
            )

        # 3. Нормализация целевых значений:
        if args_task_ == 'regression':
            print('Нормализация целевых значений: args_task_ == regression')
            # Для регрессии нормализуем целевые значения
            target_normed = normalizer.norm(target)
        else:
            print('Нормализация целевых значений else')
            # Для классификации просто изменяем форму тензора
            target_normed = target.view(-1).long()
        
        # Переносим целевые значения на GPU, если нужно
        if args_cuda_:
            print('args_cuda_')
            target_var = Variable(target_normed.cuda(non_blocking=True))
        else:
            print('args_cuda_ is None')
            target_var = Variable(target_normed)

        # 4. Прямой проход (forward pass):
        # Вычисляем выход модели для текущего батча
        output = model(*input_var)
        print('output', output)
        # Вычисляем значение функции потерь
        loss = criterion(output, target_var)
        print('loss', loss)

        # 5. Вычисление метрик:
        if args_task_ == 'regression':
            # Для регрессии:
            # Денормализуем предсказания и вычисляем MAE
            mae_error = mae(normalizer.denorm(output.data.cpu()), target)
            print('mae_error', mae_error)
            # Обновляем счетчики потерь и ошибок
            losses.update(loss.data.cpu(), target.size(0))
            mae_errors.update(mae_error, target.size(0))
        else:
            # Для классификации:
            # Вычисляем различные метрики качества
            accuracy, precision, recall, fscore, auc_score = \
                class_eval(output.data.cpu(), target)
            # Обновляем все счетчики метрик
            losses.update(loss.data.cpu().item(), target.size(0))
            accuracies.update(accuracy, target.size(0))
            precisions.update(precision, target.size(0))
            recalls.update(recall, target.size(0))
            fscores.update(fscore, target.size(0))
            auc_scores.update(auc_score, target.size(0))

        # print('Цикл 5. Вычисление метрик: завершен')

        # 6. Обратный проход (backward pass) и оптимизация:
        # Обнуляем градиенты (важно делать перед каждым backward)
        optimizer.zero_grad()
        # Вычисляем градиенты
        loss.backward()
        # Обновляем веса модели
        optimizer.step()

        # 7. Измеряем общее время обработки батча
        batch_time.update(time.time() - end)
        end = time.time()

        # 8. Периодический вывод статистики:
        if args_task_ == 'regression':
            # Вывод для регрессии
            print('Epoch: [{0}][{1}/{2}]\t'
                    'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                    'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                    'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                    'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses, mae_errors=mae_errors)
            )

In [30]:
10000 % 300

100

### def main()

In [31]:
def main():
    global args, best_mae_error

    # load data
    dataset = CIFData(*args.data_options)
    collate_fn = collate_pool
    train_loader, val_loader, test_loader = get_train_val_test_loader(
        dataset=dataset,
        collate_fn=collate_fn,
        batch_size=args.batch_size,
        train_ratio=args.train_ratio,
        num_workers=args.workers,
        val_ratio=args.val_ratio,
        test_ratio=args.test_ratio,
        pin_memory=args.cuda,
        train_size=args.train_size,
        val_size=args.val_size,
        test_size=args.test_size,
        return_test=True)

    # obtain target value normalizer
    if args.task == 'classification':
        normalizer = Normalizer(torch.zeros(2))
        normalizer.load_state_dict({'mean': 0., 'std': 1.})
    else:
        if len(dataset) < 500:
            warnings.warn('Dataset has less than 500 data points. '
                          'Lower accuracy is expected. ')
            sample_data_list = [dataset[i] for i in range(len(dataset))]
        else:
            sample_data_list = [dataset[i] for i in
                                sample(range(len(dataset)), 500)]
        _, sample_target, _ = collate_pool(sample_data_list)
        normalizer = Normalizer(sample_target)

    # build model
    structures, _, _ = dataset[0]
    orig_atom_fea_len = structures[0].shape[-1]
    nbr_fea_len = structures[1].shape[-1]
    model = CrystalGraphConvNet(orig_atom_fea_len, nbr_fea_len,
                                atom_fea_len=args.atom_fea_len,
                                n_conv=args.n_conv,
                                h_fea_len=args.h_fea_len,
                                n_h=args.n_h,
                                classification=True if args.task ==
                                                       'classification' else False)
    if args.cuda:
        model.cuda()

    # define loss func and optimizer
    if args.task == 'classification':
        criterion = nn.NLLLoss()
    else:
        criterion = nn.MSELoss()
    if args.optim == 'SGD':
        optimizer = optim.SGD(model.parameters(), args.lr,
                              momentum=args.momentum,
                              weight_decay=args.weight_decay)
    elif args.optim == 'Adam':
        optimizer = optim.Adam(model.parameters(), args.lr,
                               weight_decay=args.weight_decay)
    else:
        raise NameError('Only SGD or Adam is allowed as --optim')

    # optionally resume from a checkpoint
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            args.start_epoch = checkpoint['epoch']
            best_mae_error = checkpoint['best_mae_error']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            normalizer.load_state_dict(checkpoint['normalizer'])
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    scheduler = MultiStepLR(optimizer, milestones=args.lr_milestones,
                            gamma=0.1)

    for epoch in range(args.start_epoch, args.epochs):
        # train for one epoch
        train(train_loader, model, criterion, optimizer, epoch, normalizer)

        # evaluate on validation set
        mae_error = validate(val_loader, model, criterion, normalizer)

        if mae_error != mae_error:
            print('Exit due to NaN')
            sys.exit(1)

        scheduler.step()

        # remember the best mae_eror and save checkpoint
        if args.task == 'regression':
            is_best = mae_error < best_mae_error
            best_mae_error = min(mae_error, best_mae_error)
        else:
            is_best = mae_error > best_mae_error
            best_mae_error = max(mae_error, best_mae_error)
        save_checkpoint({
            'epoch': epoch + 1,
            'state_dict': model.state_dict(),
            'best_mae_error': best_mae_error,
            'optimizer': optimizer.state_dict(),
            'normalizer': normalizer.state_dict(),
            'args': vars(args)
        }, is_best)

    # test best model
    print('---------Evaluate Model on Test Set---------------')
    best_checkpoint = torch.load('model_best.pth.tar')
    model.load_state_dict(best_checkpoint['state_dict'])
    validate(test_loader, model, criterion, normalizer, test=True)

In [32]:
def main():
    # Объявляем глобальные переменные, которые будут использоваться в функции
    global args, best_mae_error

    # 1. ЗАГРУЗКА ДАННЫХ
    # Создаем объект dataset, используя данные из args.data_options
    dataset = CIFData(*args.data_options)
    
    # Указываем функцию для объединения данных в батчи
    collate_fn = collate_pool
    
    # Разделяем данные на обучающую, валидационную и тестовую выборки
    train_loader, val_loader, test_loader = get_train_val_test_loader(
        dataset=dataset,               # исходный датасет
        collate_fn=collate_fn,         # функция для объединения данных
        batch_size=args.batch_size,    # размер батча
        train_ratio=args.train_ratio,  # доля обучающих данных
        num_workers=args.workers,      # количество процессов для загрузки
        val_ratio=args.val_ratio,      # доля валидационных данных
        test_ratio=args.test_ratio,    # доля тестовых данных
        pin_memory=args.cuda,          # использовать ли pinned memory (если CUDA)
        train_size=args.train_size,    # абсолютный размер обучающей выборки
        val_size=args.val_size,        # абсолютный размер валидационной выборки
        test_size=args.test_size,      # абсолютный размер тестовой выборки
        return_test=True)             # возвращать ли тестовый загрузчик

    # 2. НОРМАЛИЗАЦИЯ ЦЕЛЕВЫХ ЗНАЧЕНИЙ
    if args.task == 'classification':
        # Для классификации создаем "пустой" нормализатор (не изменяет данные)
        normalizer = Normalizer(torch.zeros(2))
        normalizer.load_state_dict({'mean': 0., 'std': 1.})
    else:
        # Для регрессии предупреждаем, если данных мало
        if len(dataset) < 500:
            warnings.warn('Dataset has less than 500 data points. '
                          'Lower accuracy is expected. ')
            # Берем все данные для нормализации, если их меньше 500
            sample_data_list = [dataset[i] for i in range(len(dataset))]
        else:
            # Иначе берем случайную выборку из 500 точек
            sample_data_list = [dataset[i] for i in
                                sample(range(len(dataset)), 500)]
        
        # Получаем целевые значения для нормализации
        _, sample_target, _ = collate_pool(sample_data_list)
        normalizer = Normalizer(sample_target)

    # 3. ПОСТРОЕНИЕ МОДЕЛИ
    # Получаем структуры из первого элемента датасета
    structures, _, _ = dataset[0]
    
    # Определяем размерности входных данных
    orig_atom_fea_len = structures[0].shape[-1]  # размерность признаков атома
    nbr_fea_len = structures[1].shape[-1]        # размерность признаков соседей
    
    # Создаем модель кристаллической графовой сверточной сети
    model = CrystalGraphConvNet(
        orig_atom_fea_len,               # размерность входных признаков атома
        nbr_fea_len,                     # размерность признаков соседей
        atom_fea_len=args.atom_fea_len,  # размерность скрытых признаков атома
        n_conv=args.n_conv,              # количество сверточных слоев
        h_fea_len=args.h_fea_len,        # размерность скрытого слоя
        n_h=args.n_h,                    # количество скрытых слоев
        classification=True if args.task == 'classification' else False  # тип задачи
    )
    
    # Переносим модель на GPU, если доступно
    if args.cuda:
        model.cuda()

    # 4. ОПРЕДЕЛЕНИЕ ФУНКЦИИ ПОТЕРИ И ОПТИМИЗАТОРА
    if args.task == 'classification':
        criterion = nn.NLLLoss()  # Negative Log Likelihood Loss для классификации
    else:
        criterion = nn.MSELoss()  # Mean Squared Error Loss для регрессии
    
    # Выбираем оптимизатор
    if args.optim == 'SGD':
        optimizer = optim.SGD(
            model.parameters(), 
            args.lr,                     # скорость обучения
            momentum=args.momentum,      # момент
            weight_decay=args.weight_decay  # вес для L2-регуляризации
        )
    elif args.optim == 'Adam':
        optimizer = optim.Adam(
            model.parameters(), 
            args.lr,                     # скорость обучения
            weight_decay=args.weight_decay  # вес для L2-регуляризации
        )
    else:
        raise NameError('Only SGD or Adam is allowed as --optim')

    # 5. ЗАГРУЗКА ЧЕКПОЙНТА (ЕСЛИ НУЖНО)
    if args.resume:
        if os.path.isfile(args.resume):
            print("=> loading checkpoint '{}'".format(args.resume))
            checkpoint = torch.load(args.resume)
            
            # Восстанавливаем состояние из чекпойнта
            args.start_epoch = checkpoint['epoch']
            best_mae_error = checkpoint['best_mae_error']
            model.load_state_dict(checkpoint['state_dict'])
            optimizer.load_state_dict(checkpoint['optimizer'])
            normalizer.load_state_dict(checkpoint['normalizer'])
            
            print("=> loaded checkpoint '{}' (epoch {})"
                  .format(args.resume, checkpoint['epoch']))
        else:
            print("=> no checkpoint found at '{}'".format(args.resume))

    # 6. НАСТРОЙКА РАСПИСАНИЯ СКОРОСТИ ОБУЧЕНИЯ
    scheduler = MultiStepLR(
        optimizer, 
        milestones=args.lr_milestones,  # эпохи, когда нужно уменьшать lr
        gamma=0.1                       # множитель для уменьшения lr
    )

    # 7. ЦИКЛ ОБУЧЕНИЯ
    for epoch in range(args.start_epoch, args.epochs):
        # Обучаем модель на одной эпохе
        train(train_loader, model, criterion, optimizer, epoch, normalizer)

        # Оцениваем на валидационной выборке
        mae_error = validate(val_loader, model, criterion, normalizer)

        # Проверяем на NaN (если ошибка NaN, прекращаем выполнение)
        if mae_error != mae_error:
            print('Exit due to NaN')
            sys.exit(1)

        # Обновляем скорость обучения
        scheduler.step()

        # Сохраняем лучшую модель
        if args.task == 'regression':
            is_best = mae_error < best_mae_error  # для регрессии ищем минимум ошибки
            best_mae_error = min(mae_error, best_mae_error)
        else:
            is_best = mae_error > best_mae_error  # для классификации может быть другой критерий
            best_mae_error = max(mae_error, best_mae_error)
        
        # Сохраняем чекпойнт
        save_checkpoint({
            'epoch': epoch + 1,          # текущая эпоха
            'state_dict': model.state_dict(),  # параметры модели
            'best_mae_error': best_mae_error,  # лучшая ошибка
            'optimizer': optimizer.state_dict(),  # состояние оптимизатора
            'normalizer': normalizer.state_dict(),  # параметры нормализатора
            'args': vars(args)           # аргументы командной строки
        }, is_best)  # is_best указывает, является ли это лучшей моделью

    # 8. ТЕСТИРОВАНИЕ ЛУЧШЕЙ МОДЕЛИ
    print('---------Evaluate Model on Test Set---------------')
    # Загружаем лучшую модель
    best_checkpoint = torch.load('model_best.pth.tar')
    model.load_state_dict(best_checkpoint['state_dict'])
    # Оцениваем на тестовой выборке
    validate(test_loader, model, criterion, normalizer, test=True)

Вот подробно прокомментированный код:
Основные этапы работы функции:

1. Загрузка и подготовка данных
2. Нормализация целевых значений
3. Построение модели нейронной сети
4. Настройка функции потерь и оптимизатора
5. Загрузка чекпойнта (если требуется)
6. Настройка расписания изменения скорости обучения
7. Цикл обучения с валидацией и сохранением лучшей модели
8. Финальное тестирование на тестовом наборе данных

## Загрузка данных напрямую, без парсера

In [33]:
# import os
import csv
import random
import warnings
import functools
# import numpy as np
# import torch
# from torch.utils.data import Dataset
from pymatgen.core import Structure

 Ключевые особенности:
1. **Формат данных**:
   - Требует строгой структуры директории (`id_prop.csv`, `atom_init.json`, CIF-файлы).
   - `id_prop.csv` связывает ID кристалла с целевым свойством.

2. **Обработка кристаллов**:
   - Использует `pymatgen` для чтения CIF-файлов.
   - Автоматически находит соседей в заданном радиусе.

3. **Кэширование**:
   - Декоратор `@functools.lru_cache` ускоряет загрузку при повторных обращениях.

4. **Гибкость**:
   - Параметры `max_num_nbr` и `radius` контролируют построение графа.
   - `GaussianDistance` кодирует межатомные расстояния.

5. **Выходные данные**:
   - Возвращает признаки атомов, связи и целевое значение в формате, готовом для PyTorch.

Этот класс — ключевой компонент для преобразования кристаллических структур в графы, понятные нейросетевой модели CGCNN.

### def get_train_val_test_loader

In [34]:
from torch.utils.data.dataloader import default_collate

In [35]:
from torch.utils.data.sampler import SubsetRandomSampler

In [322]:
def get_train_val_test_loader(dataset, collate_fn=default_collate,
                            batch_size=64, train_ratio=None,
                            val_ratio=0.1, test_ratio=0.1, return_test=False,
                            num_workers=1, pin_memory=False, **kwargs):
    """
    Утилита для разделения датасета на тренировочную, валидационную и тестовую части
    и создания соответствующих DataLoader'ов.

    Важно: Датасет должен быть предварительно перемешан!

    Параметры:
    ----------
    dataset : torch.utils.data.Dataset
        Полный датасет для разделения
    collate_fn : callable, optional
        Функция для объединения выборок в батчи (по умолчанию default_collate)
    batch_size : int, optional
        Размер батча (по умолчанию 64)
    train_ratio : float, optional
        Доля тренировочных данных. Если None, вычисляется как 1 - val_ratio - test_ratio
    val_ratio : float, optional
        Доля валидационных данных (по умолчанию 0.1)
    test_ratio : float, optional
        Доля тестовых данных (по умолчанию 0.1)
    return_test : bool, optional
        Возвращать ли test_loader (по умолчанию False)
    num_workers : int, optional
        Количество подпроцессов для загрузки данных (по умолчанию 1)
    pin_memory : bool, optional
        Копировать ли данные в CUDA-pinned память (по умолчанию False)
    **kwargs : dict
        Дополнительные параметры:
        - train_size : int - абсолютный размер тренировочной выборки
        - val_size : int - абсолютный размер валидационной выборки
        - test_size : int - абсолютный размер тестовой выборки

    Возвращает:
    -------
    train_loader : DataLoader
        DataLoader для тренировочных данных
    val_loader : DataLoader
        DataLoader для валидационных данных
    test_loader : DataLoader, optional
        DataLoader для тестовых данных (только если return_test=True)
    """
    
    # Получаем общий размер датасета
    total_size = len(dataset)
    
    # Обработка размеров выборок
    if kwargs.get('train_size') is None:
        # Если размер тренировочной выборки не задан явно
        if train_ratio is None:
            # Если не задана доля тренировочных данных, вычисляем автоматически
            assert val_ratio + test_ratio < 1, "Сумма val_ratio и test_ratio должна быть < 1"
            train_ratio = 1 - val_ratio - test_ratio
            print(f'[Предупреждение] train_ratio не задан, используем 1 - val_ratio - '
                  f'test_ratio = {train_ratio} как долю тренировочных данных.')
        else:
            # Проверяем корректность заданных долей
            assert train_ratio + val_ratio + test_ratio <= 1, "Сумма долей должна быть <= 1"
    
    # Создаем список индексов [0, 1, 2, ..., total_size-1]
    indices = list(range(total_size))
    
    # Определяем размеры выборок
    # Тренировочная выборка
    train_size = kwargs.get('train_size', int(train_ratio * total_size))
    
    # Тестовая выборка
    test_size = kwargs.get('test_size', int(test_ratio * total_size))
    print('test_size', test_size)
    
    # Валидационная выборка
    valid_size = kwargs.get('val_size', int(val_ratio * total_size))
    
    # Создаем сэмплеры для разных частей датасета
    # Тренировочная выборка: первые train_size элементов
    train_sampler = SubsetRandomSampler(indices[:train_size])
    
    # Валидационная выборка: элементы между train_size и train_size+valid_size
    val_sampler = SubsetRandomSampler(
        indices[-(valid_size + test_size):-test_size] if test_size > 0 
        else indices[train_size:train_size + valid_size])
    
    # Тестовая выборка: последние test_size элементов
    if return_test:
        test_sampler = SubsetRandomSampler(indices[-test_size:])
    
    # Создаем DataLoader'ы
    train_loader = DataLoader(
        dataset, 
        batch_size=batch_size,
        sampler=train_sampler,
        num_workers=num_workers,
        collate_fn=collate_fn, 
        pin_memory=pin_memory
    )
    
    val_loader = DataLoader(
        dataset, 
        batch_size=batch_size,
        sampler=val_sampler,
        num_workers=num_workers,
        collate_fn=collate_fn, 
        pin_memory=pin_memory
    )
    
    if return_test:
        test_loader = DataLoader(
            dataset, 
            batch_size=batch_size,
            sampler=test_sampler,
            num_workers=num_workers,
            collate_fn=collate_fn, 
            pin_memory=pin_memory
        )
        return train_loader, val_loader, test_loader
    else:
        return train_loader, val_loader, None

Вот детализированный разбор функции `get_train_val_test_loader` с подробными комментариями:

Ключевые особенности:

1. **Гибкость разделения данных**:
   - Поддерживает как относительные доли (`train_ratio`), так и абсолютные размеры (`train_size`)
   - Автоматически вычисляет тренировочную долю, если не задана

2. **Сэмплеры**:
   - `SubsetRandomSampler` обеспечивает случайную выборку без дублирования
   - Четкое разделение данных без перекрытия

3. **DataLoader**:
   - Поддержка многопоточной загрузки (`num_workers`)
   - Опция `pin_memory` для ускорения передачи на GPU
   - Кастомная функция `collate_fn` для специфичных форматов данных

4. **Пример использования**:
```python
# Разделение датасета
train_loader, val_loader = get_train_val_test_loader(
    dataset=my_dataset,
    batch_size=32,
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    num_workers=4,
    pin_memory=True
)

# Или с абсолютными размерами
train_loader, val_loader, test_loader = get_train_val_test_loader(
    dataset=my_dataset,
    batch_size=32,
    train_size=8000,
    val_size=1000,
    test_size=1000,
    return_test=True
)
```

Важные замечания:
1. Датасет должен быть предварительно перемешан!
2. При использовании относительных долей убедитесь, что их сумма ≤ 1
3. Для воспроизводимости следует фиксировать random seed перед перемешиванием датасета

Эта функция стандартизирует процесс разделения данных в PyTorch-проектах, особенно полезная для задач машинного обучения с кристаллическими структурами.

Конечно! Давай разберём, что такое `default_collate` из `torch.utils.data.dataloader` и для чего он нужен.

---

 📦 `from torch.utils.data.dataloader import default_collate`

🔹 Что это?
`default_collate` — это **встроенная функция PyTorch**, которая используется по умолчанию в `DataLoader` для **объединения (collate) элементов батча**.

Когда `DataLoader` загружает батч данных (например, список из `batch_size` объектов, возвращённых `__getitem__`), он должен **объединить** их в один тензор. Именно это и делает `default_collate`.

---

 🧠 Пример

Допустим, у нас есть простой датасет:
```python
dataset = [torch.tensor([1, 2]), torch.tensor([3, 4]), torch.tensor([5, 6])]
```

Если мы хотим получить батч из трёх элементов:
```python
from torch.utils.data.dataloader import default_collate

batch = default_collate(dataset)
print(batch)
```

**Вывод:**
```python
tensor([[1, 2],
        [3, 4],
        [5, 6]])
```

➡ `default_collate` просто **объединяет список тензоров в один батч-тензор**, добавляя первую размерность — размер батча.

---

 🔄 Как работает внутри
`default_collate` умеет **рекурсивно** обрабатывать:
- тензоры (`torch.Tensor`)
- словари (`dict`)
- списки (`list`)
- кортежи (`tuple`)
- `NamedTuple`

👉 Это полезно, если каждый элемент датасета — это, например, словарь:
```python
sample = {'x': torch.tensor([1, 2]), 'y': torch.tensor([0])}
```
И `DataLoader` собирает:
```python
[
  {'x': tensor([1, 2]), 'y': tensor([0])},
  {'x': tensor([3, 4]), 'y': tensor([1])}
]
```

📦 `default_collate` превратит это в:
```python
{
  'x': tensor([[1, 2], [3, 4]]),
  'y': tensor([[0], [1]])
}
```

---

 🧩 Когда и зачем использовать явно?

Ты можешь **явно использовать `default_collate`**, если создаёшь **кастомный `collate_fn`** в `DataLoader`. Например:

```python
from torch.utils.data import DataLoader
from torch.utils.data.dataloader import default_collate

def my_collate(batch):
    # Например, можно отфильтровать или изменить элементы
    batch = [sample for sample in batch if sample is not None]
    return default_collate(batch)

loader = DataLoader(my_dataset, batch_size=4, collate_fn=my_collate)
```

---

 ✅ В контексте `torch_geometric`
В `torch_geometric`, `DataLoader` **не использует `default_collate`**, потому что графовые данные нельзя объединить простым `stack`.

Вместо этого используется `Batch.from_data_list()` — специализированный метод, который умеет собирать графы в батч (и обновлять `batch`, `edge_index`, и т.д.).

---

 📌 Вывод

| Что делает `default_collate` | Пример |
|------------------------------|--------|
| Объединяет элементы батча в тензоры | Список → батч |
| Работает с dict/list/tuple | Да |
| Используется по умолчанию в `DataLoader` | Да |
| Не используется в `torch_geometric` | Нет — у них своё объединение графов |

Если ты пишешь свои функции загрузки или фильтрации — `default_collate` может быть полезным инструментом.

Конечно! Вот краткий и понятный комментарий к строке:

```python
from torch.utils.data.sampler import SubsetRandomSampler
```

---

 📦 Что такое `SubsetRandomSampler`?

Это класс из PyTorch, который позволяет **выбирать случайные подмножества данных** из датасета **по заданным индексам**.

---

 🧠 Для чего используется?

Чтобы, например:
- Сделать **тренировочную/валидационную выборку** вручную
- Использовать **не весь датасет**, а только часть
- Загружать **разные случайные подмножества** при каждом запуске

---

 🔧 Пример использования:

```python
from torch.utils.data import DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

indices = list(range(100))  # Индексы доступных данных
train_indices = indices[:80]  # Возьмём первые 80 для обучения

sampler = SubsetRandomSampler(train_indices)

loader = DataLoader(dataset, batch_size=16, sampler=sampler)
```

🔁 **Важно**: `sampler` и `shuffle=True` несовместимы — если указываешь `sampler`, `shuffle` игнорируется.

---

✅ Когда применять?
- Когда хочешь **точно указать**, какие элементы использовать (например, валидация по индексам)
- При **кросс-валидации**
- Для **контролируемого** выбора данных

---

Если нужно — покажу, как применить `SubsetRandomSampler` для разбиения твоего датасета!

### def collate_pool(dataset_list)

In [37]:
def collate_pool(dataset_list):
    """
    Функция для объединения данных о кристаллах в батчи.
    
    Основная задача: преобразовать список отдельных кристаллов в один большой граф,
    где все атомы всех кристаллов объединены, но сохраняют принадлежность к исходным кристаллам.

    Параметры:
    ----------
    dataset_list: list of tuples
        Список кортежей для каждого кристалла в формате:
        ((atom_fea, nbr_fea, nbr_fea_idx), target, cif_id)
        
        Где:
        - atom_fea: Тензор признаков атомов [n_i, atom_fea_len]
        - nbr_fea: Тензор признаков связей [n_i, M, nbr_fea_len] 
        - nbr_fea_idx: Тензор индексов соседей [n_i, M]
        - target: Целевое значение [1, ]
        - cif_id: Идентификатор кристалла (str/int)

    Возвращает:
    -------
    Кортеж из:
    1. Подграфы для батча:
        - batch_atom_fea: Объединенные признаки атомов [N, atom_fea_len]
        - batch_nbr_fea: Объединенные признаки связей [N, M, nbr_fea_len]
        - batch_nbr_fea_idx: Объединенные индексы соседей [N, M]
        - crystal_atom_idx: Список тензоров для маппинга атомов к кристаллам
    2. batch_target: Объединенные целевые значения [N, 1]
    3. batch_cif_ids: Список идентификаторов кристаллов
    """
    
    # Инициализация пустых списков для сбора данных
    batch_atom_fea, batch_nbr_fea, batch_nbr_fea_idx = [], [], []
    crystal_atom_idx, batch_target = [], []
    batch_cif_ids = []
    base_idx = 0  # Базовый индекс для сквозной нумерации атомов

    # Обработка каждого кристалла в датасете
    for i, ((atom_fea, nbr_fea, nbr_fea_idx), target, cif_id) in enumerate(dataset_list):
        n_i = atom_fea.shape[0]  # Количество атомов в текущем кристалле

        # 1. Собираем признаки атомов
        batch_atom_fea.append(atom_fea)

        # 2. Собираем признаки связей
        batch_nbr_fea.append(nbr_fea)

        # 3. Корректируем индексы соседей с учетом базового индекса
        # (чтобы индексы были уникальными в объединенном графе)
        batch_nbr_fea_idx.append(nbr_fea_idx + base_idx)

        # 4. Создаем маппинг атомов к кристаллу
        new_idx = torch.LongTensor(np.arange(n_i) + base_idx)
        crystal_atom_idx.append(new_idx)

        # 5. Собираем целевые значения и идентификаторы
        batch_target.append(target)
        batch_cif_ids.append(cif_id)

        # 6. Увеличиваем базовый индекс на число атомов в текущем кристалле
        base_idx += n_i

    # Возвращаем объединенные данные:
    return (
        # Объединенные графовые данные:
        (
            torch.cat(batch_atom_fea, dim=0),    # Объединенные атомные признаки [N, atom_fea_len]
            torch.cat(batch_nbr_fea, dim=0),     # Объединенные признаки связей [N, M, nbr_fea_len]
            torch.cat(batch_nbr_fea_idx, dim=0),  # Объединенные индексы соседей [N, M]
            crystal_atom_idx                     # Маппинг атомов к кристаллам
        ),
        torch.stack(batch_target, dim=0),        # Объединенные целевые значения [N, 1]
        batch_cif_ids                            # Список идентификаторов
    )

 Ключевые особенности collate_pool :

1. **Объединение графов**:
   - Создает один большой граф из всех кристаллов батча
   - Сохраняет принадлежность атомов через `crystal_atom_idx`

2. **Коррекция индексов**:
   - Добавляет `base_idx` к индексам соседей, чтобы избежать коллизий
   - `base_idx` увеличивается на число атомов в каждом кристалле

3. **Структура вывода**:
   - Позволяет обрабатывать кристаллы с разным числом атомов
   - Совместима с PyTorch Geometric (аналог `Batch.from_data_list`)

4. **Эффективность**:
   - Использует `torch.cat` для быстрого объединения тензоров
   - Сохраняет исходные идентификаторы кристаллов (`cif_id`)

 Пример использования:
```python
# Предположим, dataset_list содержит 2 кристалла:
# - Кристалл 1: 3 атома
# - Кристалл 2: 5 атомов

batched_data = collate_pool(dataset_list)

# batched_data будет содержать:
# - batch_atom_fea.shape = [8, atom_fea_len] (3+5 атомов)
# - crystal_atom_idx = [tensor([0,1,2]), tensor([3,4,5,6,7])]
```

Функция критически важна для обучения CGCNN, так как преобразует список отдельных кристаллов в формат, пригодный для батч-обработки в нейросетях.

### class AtomInitializer

In [38]:
class AtomInitializer(object):
    """
    Базовый класс для инициализации векторных представлений атомов.
    
    Основная ответственность:
    - Хранение векторных представлений (эмбеддингов) для атомных типов
    - Преобразование между атомными номерами и их векторными представлениями
    
    Важно: 
    - Использовать один экземпляр AtomInitializer для каждого датасета!
    - Наследники класса должны заполнять словарь self._embedding
    
    Параметры:
    ----------
    atom_types : iterable
        Коллекция атомных номеров (например, {1, 6, 8} для H, C, O)
    """

    def __init__(self, atom_types):
        """
        Инициализация базового класса.
        
        Args:
            atom_types: множество поддерживаемых атомных номеров
        """
        self.atom_types = set(atom_types)  # Храним как множество для быстрого поиска
        self._embedding = {}  # Словарь {атомный_номер: вектор_признаков}
        self._decodedict = None  # Для обратного преобразования (инициализируется лениво)

    def get_atom_fea(self, atom_type):
        """
        Возвращает вектор признаков для указанного атомного типа.
        
        Args:
            atom_type: int (атомный номер элемента)
            
        Returns:
            Векторное представление атома
            
        Raises:
            AssertionError: если запрошен неизвестный атомный тип
        """
        assert atom_type in self.atom_types, f"Неизвестный атомный тип: {atom_type}"
        return self._embedding[atom_type]

    def load_state_dict(self, state_dict):
        """
        Загружает словарь эмбеддингов из внешнего источника.
        
        Args:
            state_dict: dict {атомный_номер: вектор_признаков}
            
        Обновляет:
            - self._embedding
            - self.atom_types
            - self._decodedict (лениво инициализируется при необходимости)
        """
        self._embedding = state_dict
        self.atom_types = set(self._embedding.keys())  # Обновляем множество атомных типов
        self._decodedict = {idx: atom_type for atom_type, idx in 
                          self._embedding.items()}  # Создаем обратный словарь

    def state_dict(self):
        """
        Возвращает текущие эмбеддинги в виде словаря.
        
        Returns:
            dict {атомный_номер: вектор_признаков}
            
        Использование:
            - Для сохранения/загрузки модели
            - Для передачи между процессами
        """
        return self._embedding

    def decode(self, idx):
        """
        Обратное преобразование: из индекса в атомный тип.
        
        Args:
            idx: индекс атомного типа в внутреннем представлении
            
        Returns:
            int: атомный номер элемента
            
        Note:
            Инициализирует _decodedict при первом вызове, если не был инициализирован
        """
        if not hasattr(self, '_decodedict') or self._decodedict is None:
            self._decodedict = {idx: atom_type for atom_type, idx in 
                              self._embedding.items()}
        return self._decodedict[idx]

Вот детализированный разбор базового класса `AtomInitializer` с подробными комментариями:

Ключевые особенности:

1. **Гибкая инициализация**:
   - Базовый класс только определяет интерфейс
   - Конкретные реализации (как `AtomCustomJSONInitializer`) заполняют `self._embedding`

2. **Безопасность**:
   - Проверка атомных типов через `assert` в `get_atom_fea`
   - Ленивая инициализация `_decodedict` для обратного преобразования

3. **Сериализация**:
   - Методы `state_dict()`/`load_state_dict()` позволяют:
     - Сохранять/загружать эмбеддинги
     - Передавать между процессами
     - Использовать в PyTorch-моделях

4. **Пример использования**:
```python
# Инициализация
initializer = AtomCustomJSONInitializer('atom_init.json')  # Наследник AtomInitializer

# Получение признаков
feats = initializer.get_atom_fea(8)  # Признаки для кислорода

# Сериализация
saved_state = initializer.state_dict()

# Десериализация
new_initializer = AtomInitializer([1, 6, 8])
new_initializer.load_state_dict(saved_state)
```

Примечания для разработчиков:
1. Наследники класса должны:
   - Заполнять `self._embedding` в своем `__init__`
   - Следить за согласованностью `self.atom_types` и ключей `self._embedding`

2. Для работы с PyTorch:
```python
# В составе Module
class MyModel(nn.Module):
    def __init__(self, atom_initializer):
        super().__init__()
        self.atom_init = atom_initializer
```

Этот класс обеспечивает единый интерфейс для работы с атомными признаками во всей системе CGCNN.

### class AtomCustomJSONInitializer

In [39]:
import json
import numpy as np

class AtomCustomJSONInitializer(AtomInitializer):
    """
    Класс для инициализации признаков атомов на основе JSON-файла.
    Создает векторные представления (эмбеддинги) для каждого типа атома.

    Параметры:
    ----------
    elem_embedding_file : str
        Путь к JSON-файлу с эмбеддингами элементов.
        Формат файла: {atomic_number: [feature1, feature2, ...], ...}

    Наследует:
    ----------
    AtomInitializer : Базовый класс для инициализации атомных признаков

    Атрибуты:
    -------
    _embedding : dict
        Словарь, где ключи - атомные номера, значения - numpy-массивы признаков
    """

    def __init__(self, elem_embedding_file):
        """
        Инициализация загрузчика атомных признаков.

        1. Загружает JSON-файл с эмбеддингами элементов
        2. Преобразует ключи в int (атомные номера)
        3. Инициализирует базовый класс с набором атомных типов
        4. Сохраняет эмбеддинги в словарь _embedding
        """
        
        # 1. Загрузка JSON-файла с эмбеддингами
        with open(elem_embedding_file) as f:
            elem_embedding = json.load(f)  # Загружаем данные из файла
        
        # 2. Преобразование ключей в int (атомные номера)
        elem_embedding = {
            int(key): value  # Преобразуем строковые ключи в целые числа
            for key, value in elem_embedding.items()
        }
        
        # 3. Получаем уникальные атомные типы
        atom_types = set(elem_embedding.keys())
        
        # 4. Инициализация родительского класса
        super(AtomCustomJSONInitializer, self).__init__(atom_types)
        
        # 5. Сохранение эмбеддингов в numpy-формате
        for key, value in elem_embedding.items():
            self._embedding[key] = np.array(value, dtype=float)

    def get_atom_fea(self, atom_type):
        """
        Возвращает вектор признаков для указанного типа атома.

        Параметры:
        ----------
        atom_type : int
            Атомный номер элемента (например, 6 для углерода)

        Возвращает:
        -------
        numpy.ndarray
            Вектор признаков для данного атома

        Вызывает:
        -------
        ValueError
            Если передан неизвестный тип атома
        """
        return super(AtomCustomJSONInitializer, self).get_atom_fea(atom_type)

### class GaussianDistance

In [40]:
import numpy as np

class GaussianDistance(object):
    """
    Класс для преобразования межатомных расстояний в гауссовы базисные функции.
    Используется для кодирования непрерывных расстояний в дискретные признаки.

    Основная идея: представить каждое расстояние как комбинацию гауссовых "колоколов",
    центрированных на определенных точках (self.filter).

    Единицы измерения: ангстремы (Å)
    """

    def __init__(self, dmin, dmax, step, var=None):
        """
        Инициализация гауссова фильтра.

        Параметры:
        ----------
        dmin : float
            Минимальное межатомное расстояние (в ангстремах)
            Пример: 0.5 Å

        dmax : float
            Максимальное межатомное расстояние (в ангстремах)
            Пример: 8.0 Å

        step : float
            Шаг между центрами гауссовых функций
            Пример: 0.2 Å

        var : float, optional
            Дисперсия (ширина) гауссовых функций. 
            Если None, используется значение step.
            Пример: 0.2
        """
        # Проверка корректности входных параметров
        assert dmin < dmax, "Минимальное расстояние должно быть меньше максимального"
        assert dmax - dmin > step, "Интервал (dmax-dmin) должен быть больше шага"

        # Создание массива центров гауссовых функций
        # np.arange создает значения от dmin до dmax с шагом step
        self.filter = np.arange(dmin, dmax + step, step)
        # Пример: при dmin=0.5, dmax=8.0, step=0.2:
        # filter = [0.5, 0.7, 0.9, ..., 8.0]

        # Установка дисперсии (ширины колоколов)
        self.var = var if var is not None else step
        # Ширина обычно равна шагу между центрами

    def expand(self, distances):
        """
        Преобразует массив расстояний в гауссовы признаки.

        Параметры:
        ----------
        distances : np.array
            Массив межатомных расстояний любой формы (например, [n_atoms, n_neighbors])
            Пример: [[1.2, 1.5], [2.1, 2.3]]

        Возвращает:
        -------
        expanded_distance : np.array
            Массив с добавленным последним измерением длины len(self.filter)
            Пример формы для входа (2,2): (2, 2, 40) (если 40 гауссовых функций)
        """
        # Добавляем новое измерение к distances для broadcasting
        # distances[..., np.newaxis] имеет форму (..., 1)
        # self.filter имеет форму (n_filters,)
        # Результат будет формы (..., n_filters)
        diff = distances[..., np.newaxis] - self.filter

        # Применяем гауссову функцию:
        # exp(-(x - μ)^2 / (2σ^2)) (здесь σ = self.var)
        gaussian = np.exp(-(diff ** 2) / (self.var ** 2))

        return gaussian

Вот детализированный разбор класса `GaussianDistance` с подробными комментариями:

 Ключевые особенности:

1. **Гауссов базис**:
   - Преобразует скалярное расстояние в вектор признаков
   - Каждый элемент вектора - значение гауссовой функции с центром в `self.filter[i]`

2. **Пример работы**:
   ```python
   gdf = GaussianDistance(dmin=0.5, dmax=8.0, step=0.2)
   distances = np.array([1.0, 2.0, 3.0])
   expanded = gdf.expand(distances)
   # expanded.shape = (3, 38)  (примерно 38 функций при таких параметрах)
   ```

3. **Параметры по умолчанию**:
   - Если `var` не указана, используется `step`
   - Это обеспечивает плавное перекрытие между соседними гауссовыми функциями

4. **Математическая формула**:
   ```
   G(d) = [exp(-(d - μ₁)²/σ²), exp(-(d - μ₂)²/σ²), ..., exp(-(d - μₙ)²/σ²)]
   где μᵢ - центры из self.filter, σ = self.var
   ```

5. **Использование в CGCNN**:
   - Преобразует межатомные расстояния в признаки связей
   - Позволяет сети учитывать не только факт связи, но и ее длину

 Визуализация гауссовых функций:
Для `dmin=0.5`, `dmax=3.0`, `step=0.5`, `var=0.5`:
```
   |
1.0|  /\      /\      /\      /\      /\      /\   
   | /  \    /  \    /  \    /  \    /  \    /  \  
0.5|/    \__/    \__/    \__/    \__/    \__/    \_
   +-----|-----|-----|-----|-----|-----|-----|-----|
   0.5  1.0  1.5  2.0  2.5  3.0
```

Этот класс критически важен для кодирования геометрической информации о кристаллической структуре в CGCNN.

### class CIFData

In [41]:
class CIFData(Dataset):
    """
    Датсет для работы с кристаллическими структурами в формате CIF-файлов.
    
    Структура директории:
    root_dir
    ├── id_prop.csv     # Свойства материалов (ID + целевое значение)
    ├── atom_init.json  # Инициализация векторов для атомов
    ├── id0.cif         # CIF-файлы структур
    ├── id1.cif
    └── ...
    
    Параметры:
    ----------
    root_dir: str       # Путь к директории с данными
    max_num_nbr: int    # Макс. число соседей в графе (по умолчанию 12)
    radius: float       # Радиус поиска соседей в ангстремах (по умолчанию 8)
    dmin: float         # Минимальное расстояние для GaussianDistance
    step: float         # Шаг дискретизации расстояний
    random_seed: int    # Seed для воспроизводимости
    """
    
    def __init__(self, root_dir, max_num_nbr=12, radius=8, dmin=0, step=0.2, random_seed=123):
        self.root_dir = root_dir
        self.max_num_nbr = max_num_nbr
        self.radius = radius
        
        # Проверка существования директории
        assert os.path.exists(root_dir), 'Директория не существует!'
        
        # Загрузка id_prop.csv (ID кристаллов и целевые значения)
        id_prop_file = os.path.join(self.root_dir, 'id_prop.csv')
        assert os.path.exists(id_prop_file), 'Файл id_prop.csv не найден!'
        with open(id_prop_file) as f:
            reader = csv.reader(f)
            self.id_prop_data = [row for row in reader]  # Список вида [[id1, target1], ...]
        
        # Перемешивание данных
        random.seed(random_seed)
        random.shuffle(self.id_prop_data)
        
        # Инициализация атомных векторов
        atom_init_file = os.path.join(self.root_dir, 'atom_init.json')
        assert os.path.exists(atom_init_file), 'Файл atom_init.json не найден!'
        self.ari = AtomCustomJSONInitializer(atom_init_file)  # Парсит JSON с атомными признаками
        
        # Инициализация преобразователя расстояний
        self.gdf = GaussianDistance(dmin=dmin, dmax=self.radius, step=step)

    def __len__(self):
        """Возвращает количество кристаллов в датасете."""
        return len(self.id_prop_data)

    @functools.lru_cache(maxsize=None)  # Кэширование загруженных структур
    def __getitem__(self, idx):
        """
        Возвращает данные для одного кристалла по индексу.
        
        Возвращает:
        --------
        (atom_fea, nbr_fea, nbr_fea_idx): tuple
            - atom_fea: Признаки атомов [n_atoms, atom_fea_len]
            - nbr_fea: Признаки связей [n_atoms, max_num_nbr, nbr_fea_len]
            - nbr_fea_idx: Индексы соседей [n_atoms, max_num_nbr]
        target: float
            Целевое значение свойства
        cif_id: str
            Идентификатор кристалла
        """
        cif_id, target = self.id_prop_data[idx]
        
        # Загрузка структуры из CIF-файла
        crystal = Structure.from_file(os.path.join(self.root_dir, cif_id + '.cif'))
        
        # Получение признаков атомов
        atom_fea = np.vstack([
            self.ari.get_atom_fea(crystal[i].specie.number)  # Вектор для атомного номера
            for i in range(len(crystal))
        ])
        
        # Поиск соседей в радиусе
        all_nbrs = crystal.get_all_neighbors(self.radius, include_index=True)
        all_nbrs = [sorted(nbrs, key=lambda x: x[1]) for nbrs in all_nbrs]  # Сортировка по расстоянию
        
        # Обработка соседей
        nbr_fea_idx, nbr_fea = [], []
        for nbr in all_nbrs:
            if len(nbr) < self.max_num_nbr:
                # Если соседей меньше max_num_nbr - заполнение нулями
                warnings.warn(f'{cif_id}: недостаточно соседей (увеличьте радиус)')
                nbr_fea_idx.append(
                    [x[2] for x in nbr] + [0] * (self.max_num_nbr - len(nbr))
                )
                nbr_fea.append(
                    [x[1] for x in nbr] + [self.radius + 1.] * (self.max_num_nbr - len(nbr))
                )
            else:
                # Берем ближайших max_num_nbr соседей
                nbr_fea_idx.append([x[2] for x in nbr[:self.max_num_nbr]])
                nbr_fea.append([x[1] for x in nbr[:self.max_num_nbr]])
        
        # Преобразование в тензоры PyTorch
        nbr_fea = self.gdf.expand(np.array(nbr_fea))  # Кодирование расстояний
        atom_fea = torch.Tensor(atom_fea)
        nbr_fea = torch.Tensor(nbr_fea)
        nbr_fea_idx = torch.LongTensor(np.array(nbr_fea_idx))
        target = torch.Tensor([float(target)])
        
        return (atom_fea, nbr_fea, nbr_fea_idx), target, cif_id

## Реализация функции main() по отдельным шагам

### 1. load data

In [42]:
# load data
# dataset = CIFData(*args.data_options)
PATH = r'C:\Users\lazarevnv\Desktop\materials_science\code_materials\graphs_code\for_cif_files'
dataset = CIFData(PATH)

In [101]:
dataset

CIFData(559)

In [102]:
type(dataset)

__main__.CIFData

In [ ]:
# проверили dataset на итерируемость
for i, val_tensor in enumerate(dataset):
    print(i)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

c:\Users\lazarevnv\Desktop\materials_science\Code_example\venv_materials_5\Lib\site-packages\pymatgen\io\cif.py:1290: UserWarning: Issues encountered while parsing CIF: 9 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


441
442
443
444
445
446
447
448
449
450
451
452
453
454
455
456
457
458
459
460
461
462
463
464
465
466
467
468
469
470
471
472
473
474
475
476
477
478
479
480
481
482
483
484
485
486
487
488
489
490
491
492
493
494
495
496
497
498
499
500
501
502
503
504
505
506
507
508
509
510
511
512
513
514
515
516
517
518
519
520
521
522
523
524
525
526
527
528
529
530
531
532
533
534
535
536
537
538
539
540
541
542
543
544
545
546
547
548
549
550
551
552
553
554
555
556
557
558


In [45]:
dataset[0]

((tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 1., 0., 0., 0., 

In [112]:
dataset[0][0][2]

tensor([[2, 3, 3, 3, 2, 2, 2, 3, 1, 1, 1, 1],
        [3, 2, 2, 2, 3, 3, 3, 2, 0, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 1, 3, 3, 3, 3],
        [1, 0, 0, 0, 1, 1, 1, 0, 2, 2, 2, 2]])

In [107]:
dataset[0][0][0].shape

torch.Size([4, 92])

In [110]:
dataset[0][0][1].shape

torch.Size([4, 12, 41])

In [111]:
dataset[0][0][2].shape

torch.Size([4, 12])

In [46]:
type(dataset[0])

tuple

In [47]:
type(dataset[0][0][0])

torch.Tensor

In [48]:
dataset[0][0][0][2], type(dataset[0][0][0][2])

(tensor([0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0.]),
 torch.Tensor)

In [49]:
dataset[0][2], type(dataset[0][2])

('865507', str)

### train_loader, val_loader, test_loader

In [133]:
collate_pool

<function __main__.collate_pool(dataset_list)>

In [132]:
collate_fn = collate_pool

In [ ]:
# train_loader, val_loader = get_train_val_test_loader(
#     dataset=dataset, # исходный датасет
#     batch_size=32, # размер батча
#     train_ratio=0.8, # доля обучающих данных
#     val_ratio=0.1, # доля валидационных данных
#     test_ratio=0.1, # доля тестовых данных
#     num_workers=4, # количество процессов для загрузки ????
#     pin_memory=True # использовать ли pinned memory (если CUDA)
# )

In [325]:
def get_train_val_test_loader(dataset, collate_fn=default_collate,
                            batch_size=64, train_ratio=None,
                            val_ratio=0.1, test_ratio=0.1, return_test=False,
                            num_workers=1, pin_memory=False, **kwargs):
    """
    Утилита для разделения датасета на тренировочную, валидационную и тестовую части
    и создания соответствующих DataLoader'ов.

    Важно: Датасет должен быть предварительно перемешан!

    Параметры:
    ----------
    dataset : torch.utils.data.Dataset
        Полный датасет для разделения
    collate_fn : callable, optional
        Функция для объединения выборок в батчи (по умолчанию default_collate)
    batch_size : int, optional
        Размер батча (по умолчанию 64)
    train_ratio : float, optional
        Доля тренировочных данных. Если None, вычисляется как 1 - val_ratio - test_ratio
    val_ratio : float, optional
        Доля валидационных данных (по умолчанию 0.1)
    test_ratio : float, optional
        Доля тестовых данных (по умолчанию 0.1)
    return_test : bool, optional
        Возвращать ли test_loader (по умолчанию False)
    num_workers : int, optional
        Количество подпроцессов для загрузки данных (по умолчанию 1)
    pin_memory : bool, optional
        Копировать ли данные в CUDA-pinned память (по умолчанию False)
    **kwargs : dict
        Дополнительные параметры:
        - train_size : int - абсолютный размер тренировочной выборки
        - val_size : int - абсолютный размер валидационной выборки
        - test_size : int - абсолютный размер тестовой выборки

    Возвращает:
    -------
    train_loader : DataLoader
        DataLoader для тренировочных данных
    val_loader : DataLoader
        DataLoader для валидационных данных
    test_loader : DataLoader, optional
        DataLoader для тестовых данных (только если return_test=True)
    """
    
    # Получаем общий размер датасета
    total_size = len(dataset)
    
    # Обработка размеров выборок
    if kwargs.get('train_size') is None:
        # Если размер тренировочной выборки не задан явно
        if train_ratio is None:
            # Если не задана доля тренировочных данных, вычисляем автоматически
            assert val_ratio + test_ratio < 1, "Сумма val_ratio и test_ratio должна быть < 1"
            train_ratio = 1 - val_ratio - test_ratio
            print(f'[Предупреждение] train_ratio не задан, используем 1 - val_ratio - '
                  f'test_ratio = {train_ratio} как долю тренировочных данных.')
        else:
            # Проверяем корректность заданных долей
            assert train_ratio + val_ratio + test_ratio <= 1, "Сумма долей должна быть <= 1"
    
    # Создаем список индексов [0, 1, 2, ..., total_size-1]
    indices = list(range(total_size))
    
    # Определяем размеры выборок
    # Тренировочная выборка
    train_size = kwargs.get('train_size', int(train_ratio * total_size))
    
    # Тестовая выборка
    test_size = kwargs.get('test_size', int(test_ratio * total_size))
    print('test_size', test_size)
    
    # Валидационная выборка
    valid_size = kwargs.get('val_size', int(val_ratio * total_size))
    
    # Создаем сэмплеры для разных частей датасета
    # Тренировочная выборка: первые train_size элементов
    train_sampler = SubsetRandomSampler(indices[:train_size])
    
    # Валидационная выборка: элементы между train_size и train_size+valid_size
    val_sampler = SubsetRandomSampler(
        indices[-(valid_size + test_size):-test_size] if test_size > 0 
        else indices[train_size:train_size + valid_size])
    
    # Тестовая выборка: последние test_size элементов
    if return_test:
        test_sampler = SubsetRandomSampler(indices[-test_size:])
        print('test_sampler', test_sampler)
    
    # Создаем DataLoader'ы
    train_loader = DataLoader(
        dataset, 
        batch_size=batch_size,
        sampler=train_sampler,
        num_workers=num_workers,
        collate_fn=collate_fn, 
        pin_memory=pin_memory
    )
    
    val_loader = DataLoader(
        dataset, 
        batch_size=batch_size,
        sampler=val_sampler,
        num_workers=num_workers,
        collate_fn=collate_fn, 
        pin_memory=pin_memory
    )
    
    if return_test:
        test_loader = DataLoader(
            dataset, 
            batch_size=batch_size,
            sampler=test_sampler,
            num_workers=num_workers,
            collate_fn=collate_fn, 
            pin_memory=pin_memory
        )
        return train_loader, val_loader, test_loader
    else:
        return train_loader, val_loader, None

In [327]:
train_loader, val_loader, test_loader = get_train_val_test_loader(
    dataset=dataset, # исходный датасет
    collate_fn=collate_fn, # функция для объединения данных
    batch_size=32, # размер батча
    train_ratio=0.8, # доля обучающих данных
    val_ratio=0.1, # доля валидационных данных
    test_ratio=0.1, # доля тестовых данных
    num_workers=0, # количество процессов для загрузки ????
    return_test=True,           # возвращать ли тестовый загрузчик
    pin_memory=False # использовать ли pinned memory (если CUDA)
)

test_size 55
test_sampler <torch.utils.data.sampler.SubsetRandomSampler object at 0x0000013FA107EF00>


In [333]:
val_loader.__dict__, test_loader.__dict__

({'dataset': CIFData(559),
  'num_workers': 0,
  'prefetch_factor': None,
  'pin_memory': False,
  'pin_memory_device': '',
  'timeout': 0,
  'worker_init_fn': None,
  '_DataLoader__multiprocessing_context': None,
  '_dataset_kind': 0,
  'batch_size': 32,
  'drop_last': False,
  'sampler': <torch.utils.data.sampler.SubsetRandomSampler at 0x13fa107f170>,
  'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x13fa107de50>,
  'generator': None,
  'collate_fn': <function __main__.collate_pool(dataset_list)>,
  'persistent_workers': False,
  '_DataLoader__initialized': True,
  '_IterableDataset_len_called': None,
  '_iterator': None},
 {'dataset': CIFData(559),
  'num_workers': 0,
  'prefetch_factor': None,
  'pin_memory': False,
  'pin_memory_device': '',
  'timeout': 0,
  'worker_init_fn': None,
  '_DataLoader__multiprocessing_context': None,
  '_dataset_kind': 0,
  'batch_size': 32,
  'drop_last': False,
  'sampler': <torch.utils.data.sampler.SubsetRandomSampler at 0x13fa107ef00>

In [ ]:
test_loader = DataLoader(
    dataset, 
    batch_size=32,
    sampler=test_sampler,
    num_workers=0,
    collate_fn=collate_fn, 
    pin_memory=pin_memory
)

In [321]:
test_loader

In [51]:
# def main():
#     # Объявляем глобальные переменные, которые будут использоваться в функции
#     global args, best_mae_error

#     # 1. ЗАГРУЗКА ДАННЫХ
#     # Создаем объект dataset, используя данные из args.data_options
#     dataset = CIFData(*args.data_options)
    
#     # Указываем функцию для объединения данных в батчи
#     collate_fn = collate_pool
    
#     # Разделяем данные на обучающую, валидационную и тестовую выборки
#     train_loader, val_loader, test_loader = get_train_val_test_loader(
#         dataset=dataset,               # исходный датасет
#         collate_fn=collate_fn,         # функция для объединения данных
#         batch_size=args.batch_size,    # размер батча
#         train_ratio=args.train_ratio,  # доля обучающих данных
#         num_workers=args.workers,      # количество процессов для загрузки
#         val_ratio=args.val_ratio,      # доля валидационных данных
#         test_ratio=args.test_ratio,    # доля тестовых данных
#         pin_memory=args.cuda,          # использовать ли pinned memory (если CUDA)
#         train_size=args.train_size,    # абсолютный размер обучающей выборки
#         val_size=args.val_size,        # абсолютный размер валидационной выборки
#         test_size=args.test_size,      # абсолютный размер тестовой выборки
#         return_test=True)             # возвращать ли тестовый загрузчик

In [ ]:
# смотрим перечень атрибутов torch.utils.data.dataloader.DataLoader
dir(train_loader)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_index_sampler',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_init_fn']

In [129]:
train_loader.pin_memory

True

In [137]:
train_loader.__dict__

{'dataset': CIFData(559),
 'num_workers': 0,
 'prefetch_factor': None,
 'pin_memory': False,
 'pin_memory_device': '',
 'timeout': 0,
 'worker_init_fn': None,
 '_DataLoader__multiprocessing_context': None,
 '_dataset_kind': 0,
 'batch_size': 32,
 'drop_last': False,
 'sampler': <torch.utils.data.sampler.SubsetRandomSampler at 0x13fa0ed32c0>,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x13fa0a0b680>,
 'generator': None,
 'collate_fn': <function __main__.collate_pool(dataset_list)>,
 'persistent_workers': False,
 '_DataLoader__initialized': True,
 '_IterableDataset_len_called': None,
 '_iterator': None}

In [118]:
train_loader.__class__

torch.utils.data.dataloader.DataLoader

In [117]:
train_loader.__annotations__

{'dataset': torch.utils.data.dataset.Dataset[+T_co],
 'batch_size': typing.Optional[int],
 'num_workers': int,
 'pin_memory': bool,
 'drop_last': bool,
 'timeout': float,
 'sampler': typing.Union[torch.utils.data.sampler.Sampler, typing.Iterable],
 'pin_memory_device': str,
 'prefetch_factor': typing.Optional[int],
 '_iterator': typing.Optional[ForwardRef('_BaseDataLoaderIter')]}

In [141]:
batch = next(iter(train_loader))
# print(batch[0].is_pinned())  # Должно быть True

In [156]:
type(batch[0])

tuple

In [155]:
for i, val in enumerate(batch[0]):
    print(i)

0
1
2
3


In [150]:
batch[1]

tensor([[-2.2187],
        [-2.8266],
        [-0.3840],
        [-2.7048],
        [ 1.0462],
        [-0.8626],
        [-0.6683],
        [ 0.8619],
        [-2.6656],
        [-0.1846],
        [-0.8001],
        [-0.4477],
        [-0.0929],
        [-0.7488],
        [ 1.1629],
        [ 0.0201],
        [ 0.7485],
        [-0.7449],
        [ 0.5064],
        [-3.3070],
        [-0.7734],
        [-0.4111],
        [-0.4184],
        [-0.3223],
        [-0.1731],
        [-0.4566],
        [-0.3390],
        [-2.6270],
        [-0.3279],
        [-0.2903],
        [-0.7006],
        [-0.6737]])

In [334]:
len(batch), type(batch)

(3, tuple)

In [160]:
train_loader

In [138]:
sample = [dataset[i] for i in range(2)]
print(train_loader.collate_fn(sample))  # Должен вернуть батч

((tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 

In [114]:
type(train_loader)

torch.utils.data.dataloader.DataLoader

Вот подробный разбор содержимого `train_loader.__dict__` для `DataLoader` в PyTorch:

---

**1. Основные параметры загрузки данных**
```python
'dataset': CIFData(559)  # Объект датасета (в вашем случае CIFData с 559 образцами)
'num_workers': 4         # Количество процессов для загрузки данных (ускоряет загрузку)
'prefetch_factor': 2     # Сколько батчей каждый worker загружает заранее
'pin_memory': True       # Копировать данные в pinned memory (ускоряет перенос на GPU)
'pin_memory_device': ''  # Устройство для pin_memory (пусто = автоматически)
'timeout': 0             # Максимальное время ожидания данных от worker (0 = бесконечно)
```

---

 **2. Стратегии выборки (sampling)**
```python
'sampler': SubsetRandomSampler  # Стратегия выборки элементов (в вашем случае случайная из подмножества)
'batch_sampler': BatchSampler   # Формирует батчи из индексов, полученных от sampler
'drop_last': False             # Не отбрасывать последний неполный батч (если len(dataset) % batch_size != 0)
'batch_size': 32               # Размер батча
```

---

 **3. Функции обработки данных**
```python
'collate_fn': default_collate  # Функция для объединения отдельных элементов в батч
'worker_init_fn': None        # Функция инициализации worker-процессов (не используется)
```

---

 **4. Внутренние служебные поля**
```python
'_DataLoader__multiprocessing_context': None  # Контекст для многопроцессорности (None = стандартный)
'_dataset_kind': 0          # Тип датасета (0 = обычный Dataset, 1 = IterableDataset)
'generator': None           # Генератор для случайных операций (если используется)
'persistent_workers': False # Сохранять ли workers между эпохами (экономит время на создание процессов)
'_DataLoader__initialized': True  # Флаг инициализации DataLoader
'_IterableDataset_len_called': None  # Для IterableDataset
'_iterator': None           # Текущий активный итератор (None = не создан)
```

---

 **Подробное объяснение ключевых компонентов**

 **1. Dataset (`CIFData(559)`)**
- **Что это**: Ваш кастомный датасет, содержащий 559 образцов.
- **Как проверить**:
  ```python
  print(train_loader.dataset[0])  # Посмотреть первый элемент
  print(len(train_loader.dataset))  # Проверить размер (должно быть 559)
  ```

 **2. Multi-processing (`num_workers=4`)**
- **Зачем**: Параллельная загрузка данных (4 процесса).
- **Проблемы**: Если данные загружаются медленно:
  ```python
  # Временное отключение для диагностики
  train_loader = DataLoader(..., num_workers=0)
  ```

 **3. Batch Formation (`batch_sampler`, `collate_fn`)**
- **Как работает**:
  1. `sampler` выбирает индексы (например, `[3, 1, 4]`).
  2. `batch_sampler` группирует их в батчи (`[[3,1,4], [5,2,6], ...]`).
  3. `collate_fn` объединяет данные по этим индексам в тензоры.

- **Пример для `default_collate`**:
  ```python
  batch = [dataset[i] for i in [0,1,2]]  # 3 отдельных элемента
  print(train_loader.collate_fn(batch))   # Объединенный батч
  ```

 **4. Pinned Memory (`pin_memory=True`)**
- **Зачем**: Ускоряет копирование данных на GPU (используется `cuda()`).
- **Проверка**:
  ```python
  batch = next(iter(train_loader))
  print(batch[0].is_pinned())  # Должно быть True
  ```

---

 **Как диагностировать проблемы**

 **Если загрузка зависает**:
1. Проверьте датасет без `DataLoader`:
   ```python
   # Работает ли датасет напрямую?
   for i in range(3):
       print(dataset[i])  # Не должно зависать
   ```

2. Тест `collate_fn`:
   ```python
   sample = [dataset[i] for i in range(2)]
   print(train_loader.collate_fn(sample))  # Должен вернуть батч
   ```

3. Отключите многопоточность:
   ```python
   test_loader = DataLoader(dataset, num_workers=0)
   next(iter(test_loader))  # Проверяем работу
   ```

---

 **Вывод**
Ваш `DataLoader` настроен для:
- Параллельной загрузки (4 workers)
- Случайной выборки (`SubsetRandomSampler`)
- Оптимизированной передачи на GPU (`pin_memory=True`)

Если есть проблемы (например, зависание), начинайте диагностику с:
1. Проверки датасета (`dataset[0]`)
2. Отключения `num_workers`
3. Проверки `collate_fn`.

In [53]:

# # collate_fn = collate_pool
# train_loader, val_loader, test_loader = get_train_val_test_loader(
#     dataset=dataset,
#     collate_fn=collate_pool,
#     batch_size=args.batch_size,
#     train_ratio=args.train_ratio,
#     num_workers=args.workers,
#     val_ratio=args.val_ratio,
#     test_ratio=args.test_ratio,
#     pin_memory=args.cuda,
#     train_size=args.train_size,
#     val_size=args.val_size,
#     test_size=args.test_size,
#     return_test=True)

**`train_loader` — это не часть датасета, а "инструмент" для загрузки данных из датасета батчами.** Разберём подробно:

---

 **1. Что такое `train_loader`?**
Это объект класса `torch.utils.data.DataLoader`, который:
- **Не содержит данные напрямую** — он лишь организует доступ к ним через датасет (`dataset`).
- **Автоматически разбивает данные на батчи** (пачки фиксированного размера).
- **Поддерживает многопоточную загрузку** (если `num_workers > 0`).
- **Может перемешивать данные** (если `shuffle=True`).

```python
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset,          # Ваш датасет (например, CIFData)
    batch_size=32,    # Размер батча
    shuffle=True,     # Перемешивание данных
    num_workers=4     # Число процессов для загрузки
)
```

---

 **2. Как связаны `train_loader` и `dataset`?**
| **`dataset`** | **`train_loader`** |
|--------------|-------------------|
| Содержит **все данные** (например, 559 образцов в вашем случае). | Содержит **метаданные** о том, как загружать данные из `dataset`. |
| Возвращает **один элемент** при обращении по индексу (`dataset[i]`). | Возвращает **пачку элементов (батч)** при итерации (`next(iter(train_loader))`). |
| Пример: `CIFData(559)` | Пример: `DataLoader(dataset=CIFData(559), batch_size=32)` → 18 батчей (559/32=17.46 → 18 с учётом `drop_last=False`). |

---

 **3. Как проверить содержимое?**
 Для датасета:
```python
print(len(dataset))      # Общее число образцов (например, 559)
print(dataset[0])        # Первый образец (зависит от вашего CIFData)
```

 Для DataLoader:
```python
print(len(train_loader))  # Число батчей (например, 18 при batch_size=32)
batch = next(iter(train_loader))
print(batch[0].shape)    # Размерность входных данных в батче (например, [32, ...])
```

---

 **4. Почему `train_loader` не является частью датасета?**
- **Данные физически хранятся в `dataset`** — `train_loader` лишь управляет доступом к ним.
- **Один датасет можно использовать в нескольких `DataLoader`** с разными параметрами:
  ```python
  train_loader = DataLoader(dataset, batch_size=32, shuffle=True)
  val_loader = DataLoader(dataset, batch_size=16, shuffle=False)
  ```

---

 **5. Аналогия из реального мира**
| **Концепция** | **Аналог** |
|--------------|-----------|
| `dataset` | Библиотека с книгами (все данные). |
| `train_loader` | Библиотекарь, который: <br> - Приносит книги пачками (`batch_size`), <br> - Может перемешивать порядок (`shuffle`), <br> - Работает с помощниками (`num_workers`). |

---

 **6. Что возвращает итерация по `train_loader`?**
Структура батча зависит от того, что возвращает ваш `dataset.__getitem__()`. Например:
- Если `dataset[i]` возвращает `(input, target, metadata)`, то:
  ```python
  for batch in train_loader:
      inputs, targets, _ = batch  # inputs.shape = [32, ...], targets.shape = [32]
  ```
- Если `dataset[i]` возвращает `dict`, то `batch` будет словарём.

Проверьте структуру вашего датасета:
```python
sample = dataset[0]
print("Элемент датасета:", type(sample), sample)
```

---

 **Вывод**
`train_loader` — это **"интеллектуальный обёрточный слой"** для датасета, который:
1. Не дублирует данные.
2. Управляет батчированием, перемешиванием и многопоточной загрузкой.
3. Позволяет итерироваться по данным в удобном для обучения формате.

Если нужно работать с сырыми данными — обращайтесь напрямую к `dataset`. Для обучения моделей используйте `train_loader`.

### 2. obtain target value normalizer  НОРМАЛИЗАЦИЯ ЦЕЛЕВЫХ ЗНАЧЕНИЙ

In [54]:
import warnings
from random import sample

In [55]:
# args_task_ = 'classification'
args_task_ = 'regression'

In [56]:
# obtain target value normalizer
# if args.task == 'classification':
if args_task_ == 'classification':
    # Для классификации создаем "пустой" нормализатор (не изменяет данные)
    normalizer = Normalizer(torch.zeros(2))
    normalizer.load_state_dict({'mean': 0., 'std': 1.})
else:
    # Для регрессии предупреждаем, если данных мало
    if len(dataset) < 500:
        warnings.warn('Dataset has less than 500 data points. '
                        'Lower accuracy is expected. ') # Набор данных содержит менее 500 точек данных, ожидается более низкая точность
        # Берем все данные для нормализации, если их меньше 500
        sample_data_list = [dataset[i] for i in range(len(dataset))]
    else:
        # Иначе берем случайную выборку из 500 точек
        sample_data_list = [dataset[i] for i in
                            sample(range(len(dataset)), 500)]
        
    # Получаем целевые значения для нормализации
    _, sample_target, _ = collate_pool(sample_data_list)
    normalizer = Normalizer(sample_target)

C:\Users\lazarevnv\AppData\Local\Temp\ipykernel_23908\247230285.py:90: UserWarning: 1208620: недостаточно соседей (увеличьте радиус)
  warnings.warn(f'{cif_id}: недостаточно соседей (увеличьте радиус)')
c:\Users\lazarevnv\Desktop\materials_science\Code_example\venv_materials_5\Lib\site-packages\pymatgen\io\cif.py:1290: UserWarning: Issues encountered while parsing CIF: 3 fractional coordinates rounded to ideal values to avoid issues with finite precision.
  warnings.warn("Issues encountered while parsing CIF: " + "\n".join(self.warnings))


In [57]:
   
    # if args.task == 'classification':
    #     # Для классификации создаем "пустой" нормализатор (не изменяет данные)
    #     normalizer = Normalizer(torch.zeros(2))
    #     normalizer.load_state_dict({'mean': 0., 'std': 1.})
    # else:
    #     # Для регрессии предупреждаем, если данных мало
    #     if len(dataset) < 500:
    #         warnings.warn('Dataset has less than 500 data points. '
    #                       'Lower accuracy is expected. ')
    #         # Берем все данные для нормализации, если их меньше 500
    #         sample_data_list = [dataset[i] for i in range(len(dataset))]
    #     else:
    #         # Иначе берем случайную выборку из 500 точек
    #         sample_data_list = [dataset[i] for i in
    #                             sample(range(len(dataset)), 500)]
        
    #     # Получаем целевые значения для нормализации
    #     _, sample_target, _ = collate_pool(sample_data_list)
    #     normalizer = Normalizer(sample_target)

In [58]:
len(dataset)

559

### 3. build model ПОСТРОЕНИЕ МОДЕЛИ

In [59]:
# build model
# Получаем структуры из первого элемента датасета
structures, _, _ = dataset[0] # что это такое _, _

In [60]:
dataset[0], len(dataset[0])

(((tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
            0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
            0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
            0., 0.],
           [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
            0., 0., 0., 0., 0., 1.

In [61]:
structures, len(structures)

((tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 1., 0., 0., 0., 

In [62]:
structures[0].shape

torch.Size([4, 92])

In [63]:
_

'865507'

In [64]:
dataset[0][0][0], len(dataset[0][0][0])

(tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
          0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
          0., 0.],
         [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
          0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1.

In [65]:
dataset[0][0][0][0], len(dataset[0][0][0][0])

(tensor([0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
         0., 0.]),
 92)

In [66]:
structures[1].shape

torch.Size([4, 12, 41])

In [67]:
# Определяем размерности входных данных
orig_atom_fea_len = structures[0].shape[-1] # размерность признаков атома
nbr_fea_len = structures[1].shape[-1]        # размерность признаков соседей

In [68]:
orig_atom_fea_len, nbr_fea_len

(92, 41)

In [69]:
# Создаем модель кристаллической графовой сверточной сети
model = CrystalGraphConvNet(
    orig_atom_fea_len,               # размерность входных признаков атома
    nbr_fea_len,                     # размерность признаков соседей
    # atom_fea_len=args.atom_fea_len,  # размерность скрытых признаков атома
    atom_fea_len = 64,  # размерность скрытых признаков атома /# Размер атомных эмбеддингов
    # n_conv=args.n_conv,              # количество сверточных слоев
    n_conv = 3,              # количество сверточных слоев
    # h_fea_len=args.h_fea_len,        # размерность скрытого слоя
    h_fea_len=128,        # размерность скрытого слоя
    # n_h=args.n_h,                    # количество скрытых слоев
    n_h=3,                    # количество скрытых слоев
    # classification=True if args.task == 'classification' else False  # тип задачи
    classification=True if args_task_ == 'classification' else False  # тип задачи
)

In [70]:
    # # ПОСТРОЕНИЕ МОДЕЛИ
    # # Получаем структуры из первого элемента датасета
    # structures, _, _ = dataset[0]
    
    # # Определяем размерности входных данных
    # orig_atom_fea_len = structures[0].shape[-1]  # размерность признаков атома
    # nbr_fea_len = structures[1].shape[-1]        # размерность признаков соседей
    
    # # Создаем модель кристаллической графовой сверточной сети
    # model = CrystalGraphConvNet(
    #     orig_atom_fea_len,               # размерность входных признаков атома
    #     nbr_fea_len,                     # размерность признаков соседей
    #     atom_fea_len=args.atom_fea_len,  # размерность скрытых признаков атома
    #     n_conv=args.n_conv,              # количество сверточных слоев
    #     h_fea_len=args.h_fea_len,        # размерность скрытого слоя
    #     n_h=args.n_h,                    # количество скрытых слоев
    #     classification=True if args.task == 'classification' else False  # тип задачи
    # )

In [71]:
# # Переносим модель на GPU, если доступно
# if args.cuda:
#     print('args.cuda')
#     model.cuda()
# else:
#     print('else')

### 4. ОПРЕДЕЛЕНИЕ ФУНКЦИИ ПОТЕРИ И ОПТИМИЗАТОРА

In [72]:
# 4. ОПРЕДЕЛЕНИЕ ФУНКЦИИ ПОТЕРИ И ОПТИМИЗАТОРА
if args_task_ == 'classification':
    criterion = nn.NLLLoss()  # Negative Log Likelihood Loss для классификации
else:
    criterion = nn.MSELoss()  # Mean Squared Error Loss для регрессии

In [73]:
criterion

MSELoss()

In [74]:
    # # 4. ОПРЕДЕЛЕНИЕ ФУНКЦИИ ПОТЕРИ И ОПТИМИЗАТОРА
    # if args.task == 'classification':
    #     criterion = nn.NLLLoss()  # Negative Log Likelihood Loss для классификации
    # else:
    #     criterion = nn.MSELoss()  # Mean Squared Error Loss для регрессии

In [75]:
# Параметры обучения
args_lr_ = 0.01            # Скорость обучения
epochs = 60          # Число эпох
args_optim_ = 'Adam'
args_momentum_ = None # при SGD нужно задать этот параметр
args_weight_decay_ = 0 # поробуем 0

In [76]:
# Выбираем оптимизатор
if args_optim_ == 'SGD':
    optimizer = optim.SGD(
        model.parameters(), 
        args_lr_,                     # скорость обучения
        momentum=args_momentum_,      # момент
        weight_decay=args_weight_decay_  # вес для L2-регуляризации
    )
elif args_optim_ == 'Adam':
    optimizer = optim.Adam(
        model.parameters(), 
        args_lr_,                     # скорость обучения
        weight_decay=args_weight_decay_  # вес для L2-регуляризации
    )
else:
    raise NameError('Only SGD or Adam is allowed as --optim')

In [77]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [78]:
    # # Выбираем оптимизатор
    # if args.optim == 'SGD':
    #     optimizer = optim.SGD(
    #         model.parameters(), 
    #         args.lr,                     # скорость обучения
    #         momentum=args.momentum,      # момент
    #         weight_decay=args.weight_decay  # вес для L2-регуляризации
    #     )
    # elif args.optim == 'Adam':
    #     optimizer = optim.Adam(
    #         model.parameters(), 
    #         args.lr,                     # скорость обучения
    #         weight_decay=args.weight_decay  # вес для L2-регуляризации
    #     )
    # else:
    #     raise NameError('Only SGD or Adam is allowed as --optim')

### 5. ЗАГРУЗКА ЧЕКПОЙНТА (ЕСЛИ НУЖНО)

пока неясно что это и зачем

In [79]:
# checkpoint['epoch']

In [80]:
# # 5. ЗАГРУЗКА ЧЕКПОЙНТА (ЕСЛИ НУЖНО)
# if args.resume:
#     if os.path.isfile(args.resume):
#         print("=> loading checkpoint '{}'".format(args.resume))
#         checkpoint = torch.load(args.resume)
        
#         # Восстанавливаем состояние из чекпойнта
#         args.start_epoch = checkpoint['epoch']
#         best_mae_error = checkpoint['best_mae_error']
#         model.load_state_dict(checkpoint['state_dict'])
#         optimizer.load_state_dict(checkpoint['optimizer'])
#         normalizer.load_state_dict(checkpoint['normalizer'])
        
#         print("=> loaded checkpoint '{}' (epoch {})"
#                 .format(args.resume, checkpoint['epoch']))
#     else:
#         print("=> no checkpoint found at '{}'".format(args.resume))

In [365]:
if args_resume_:
    print(1)

NameError: name 'args_resume_' is not defined

In [ ]:
# 5. ЗАГРУЗКА ЧЕКПОЙНТА (ЕСЛИ НУЖНО)
if args.resume:
    if os.path.isfile(args.resume):
        print("=> loading checkpoint '{}'".format(args.resume))
        checkpoint = torch.load(args.resume)
        
        # Восстанавливаем состояние из чекпойнта
        args.start_epoch = checkpoint['epoch']
        best_mae_error = checkpoint['best_mae_error']
        model.load_state_dict(checkpoint['state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer'])
        normalizer.load_state_dict(checkpoint['normalizer'])
        
        print("=> loaded checkpoint '{}' (epoch {})"
                .format(args.resume, checkpoint['epoch']))
    else:
        print("=> no checkpoint found at '{}'".format(args.resume))

### 6. НАСТРОЙКА РАСПИСАНИЯ СКОРОСТИ ОБУЧЕНИЯ

Пока не понятно, что это такое

In [82]:
# 6. НАСТРОЙКА РАСПИСАНИЯ СКОРОСТИ ОБУЧЕНИЯ
scheduler = MultiStepLR(
    optimizer, 
    # milestones=args.lr_milestones,  # эпохи, когда нужно уменьшать lr
    milestones=[10],  # эпохи, когда нужно уменьшать lr
    gamma=0.1                       # множитель для уменьшения lr
)

### 7. ЦИКЛ ОБУЧЕНИЯ

In [83]:
import sys

In [84]:
# sys.exit(1)

In [85]:
args_epochs_ = 60 # число эпох
args_start_epoch_ = 0

In [86]:
for epoch in range(args_start_epoch_, args_epochs_):
    print(epoch)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59


**Ошибка в вашем предположении:** `DataLoader` (в вашем случае `train_loader`) **является итерируемым объектом**, но не является *индексируемым* (то есть не поддерживает операцию `train_loader[0]`). Это разные понятия!

---

 **Как правильно итерироваться по `train_loader`**

 1. **Базовый вариант (если `batch = (input, target, metadata)`)**
```python
from tqdm import tqdm

for batch in tqdm(train_loader):  # Автоматически создаёт итератор
    input, target, _ = batch      # Распаковываем батч
    # Ваш код обработки
    print(input.shape, target.shape)
```

 2. **С доступом к индексу батча (если нужен `i`)**
```python
for i, (input, target, _) in tqdm(enumerate(train_loader)):
    # Ваш код
    print(f"Batch {i}: Input={input.shape}, Target={target.shape}")
```

 3. **Если возникает ошибка распаковки**
Проверьте структуру `batch` и подстройте распаковку:
```python
first_batch = next(iter(train_loader))
print("Структура батча:", type(first_batch), [type(x) for x in first_batch])

# Пример подстройки под ваши данные
for batch in tqdm(train_loader):
    if len(batch) == 3:
        input, target, _ = batch
    elif len(batch) == 2:
        input, target = batch
    else:
        raise ValueError(f"Неожиданная структура батча: {batch}")
```

---

 **Почему это работает?**
- **Итерируемость**: `DataLoader` реализует метод `__iter__()`, который возвращает итератор.
- **Механика работы**:
  1. При вызове `for batch in train_loader` скрыто создаётся итератор через `iter(train_loader)`.
  2. На каждой итерации вызывается `next()`, который:
     - Выбирает индексы для батча (через `sampler`),
     - Загружает соответствующие данные из `dataset`,
     - Применяет `collate_fn` для объединения в тензоры.

---

 **Что делать, если цикл не запускается?**
1. **Проверьте длину `train_loader`**:
   ```python
   print("Количество батчей:", len(train_loader))  # Должно быть > 0
   ```

2. **Проверьте первый батч вручную**:
   ```python
   try:
       batch = next(iter(train_loader))
       print("Первый батч успешно загружен!")
   except Exception as e:
       print("Ошибка загрузки:", e)
   ```

3. **Убедитесь, что датасет не пуст**:
   ```python
   print("Размер датасета:", len(train_loader.dataset))
   print("Первый элемент датасета:", train_loader.dataset[0])
   ```

---

 **Пример полного рабочего кода**
```python
from torch.utils.data import DataLoader, TensorDataset
import torch
from tqdm import tqdm

# Создаём тестовый датасет
data = torch.randn(100, 3)  # 100 samples, 3 features
targets = torch.randint(0, 2, (100,))
dataset = TensorDataset(data, targets)

# Создаём DataLoader
train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Итерируемся
for i, (inputs, labels) in tqdm(enumerate(train_loader), total=len(train_loader)):
    print(f"Batch {i}: inputs={inputs.shape}, labels={labels.shape}")
    # Ваш код обучения...
```

---

 **Ключевые выводы**
1. `train_loader` — **итерируемый**, но не индексируемый.
2. Используйте `for batch in train_loader` или `enumerate(train_loader)`.
3. Если цикл не работает — проверьте:
   - Не пуст ли `dataset`,
   - Корректность `collate_fn`,
   - Размеры данных в батче.

Для вашего случая с `(input, target, _)` подойдёт вариант 1 или 2 из примеров выше.

### def train с корректировками

In [340]:
train_loader.batch_size

32

In [346]:
32*14

448

In [345]:
len(train_loader)

14

In [344]:
train_loader.__dict__

{'dataset': CIFData(559),
 'num_workers': 0,
 'prefetch_factor': None,
 'pin_memory': False,
 'pin_memory_device': '',
 'timeout': 0,
 'worker_init_fn': None,
 '_DataLoader__multiprocessing_context': None,
 '_dataset_kind': 0,
 'batch_size': 32,
 'drop_last': False,
 'sampler': <torch.utils.data.sampler.SubsetRandomSampler at 0x13fa107d4c0>,
 'batch_sampler': <torch.utils.data.sampler.BatchSampler at 0x13fa064ba10>,
 'generator': None,
 'collate_fn': <function __main__.collate_pool(dataset_list)>,
 'persistent_workers': False,
 '_DataLoader__initialized': True,
 '_IterableDataset_len_called': None,
 '_iterator': None}

In [339]:
dir(train_loader)

['_DataLoader__initialized',
 '_DataLoader__multiprocessing_context',
 '_IterableDataset_len_called',
 '__annotations__',
 '__class__',
 '__class_getitem__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__orig_bases__',
 '__parameters__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_auto_collation',
 '_dataset_kind',
 '_get_iterator',
 '_index_sampler',
 '_iterator',
 'batch_sampler',
 'batch_size',
 'check_worker_number_rationality',
 'collate_fn',
 'dataset',
 'drop_last',
 'generator',
 'multiprocessing_context',
 'num_workers',
 'persistent_workers',
 'pin_memory',
 'pin_memory_device',
 'prefetch_factor',
 'sampler',
 'timeout',
 'worker_init_fn']

In [347]:
n = 0
for i, batch in tqdm(enumerate(train_loader)):
    print(i)
    n+=1
    # print(batch)

14it [00:00, 274.50it/s]

0
1
2
3
4
5
6
7
8
9
10
11
12
13


In [356]:
def train(train_loader, model, criterion, optimizer, epoch, normalizer):
    """
    Функция для обучения модели на одной эпохе
    Параметры:
        train_loader - DataLoader для обучающих данных
        model - модель нейронной сети
        criterion - функция потерь
        optimizer - оптимизатор
        epoch - номер текущей эпохи
        normalizer - нормализатор для целевых значений
    """
    
    # Инициализация счетчиков для мониторинга процесса обучения:
    
    # batch_time - время обработки одного батча
    batch_time = AverageMeter()
    print('batch_time', batch_time)
    # data_time - время загрузки данных
    data_time = AverageMeter()
    print('data_time', data_time)
    # losses - значения функции потерь
    losses = AverageMeter()
    print('losses', losses)
    
    # В зависимости от типа задачи инициализируем соответствующие метрики:
    if args_task_ == 'regression':
        # Для регрессии используем MAE (Mean Absolute Error)
        mae_errors = AverageMeter()
        print('mae_errors', mae_errors)
    else:
        # Для классификации используем несколько метрик:
        accuracies = AverageMeter()    # Точность
        precisions = AverageMeter()    # Прецизионность
        recalls = AverageMeter()       # Полнота
        fscores = AverageMeter()       # F1-мера
        auc_scores = AverageMeter()    # AUC-ROC

    # Переводим модель в режим обучения (это важно для слоев типа Dropout и BatchNorm)
    model.train()
    print('Сработало model.train()')

    # Фиксируем время начала обработки
    end = time.time()
    print('end', end)
    
    # Основной цикл обучения по батчам:
    print('Начинаем цикл обучения по батчам!')
    for i, (batch) in tqdm(enumerate(train_loader)):
        input = batch[0]
        target = batch[1]
        print('Новый i-й шаг цикла:', i)
        # print('input - входные данные модели', input)
        # print('target - целевые значения', target)

        # input - входные данные модели
        # target - целевые значения
        # _ - дополнительные метаданные (не используются)
        
        # 1. Измеряем время загрузки данных
        data_time.update(time.time() - end)
        print('Измеряем время загрузки данных')

        # 2. Подготовка входных данных:
        # Переносим данные на GPU, если используется CUDA
        if args_cuda_:
            print('args_cuda_')
            input_var = (
                Variable(input[0].cuda(non_blocking=True)),  # Атомные признаки
                Variable(input[1].cuda(non_blocking=True)),  # Признаки соседей
                input[2].cuda(non_blocking=True),           # Индексы соседей
                [crys_idx.cuda(non_blocking=True) for crys_idx in input[3]]  # Индексы кристаллов
            )
        else:
            print('args_cuda_ else')
            input_var = (
                Variable(input[0]),
                Variable(input[1]),
                input[2],
                input[3]
            )

        # 3. Нормализация целевых значений:
        if args_task_ == 'regression':
            print('Нормализация целевых значений: args_task_ == regression')
            # Для регрессии нормализуем целевые значения
            target_normed = normalizer.norm(target)
        else:
            print('Нормализация целевых значений else')
            # Для классификации просто изменяем форму тензора
            target_normed = target.view(-1).long()
        
        # Переносим целевые значения на GPU, если нужно
        if args_cuda_:
            print('args_cuda_')
            target_var = Variable(target_normed.cuda(non_blocking=True))
        else:
            print('args_cuda_ is None')
            target_var = Variable(target_normed)

        # 4. Прямой проход (forward pass):
        # Вычисляем выход модели для текущего батча
        output = model(*input_var)
        print('output', output)
        # Вычисляем значение функции потерь
        loss = criterion(output, target_var)
        print('loss', loss)

        # 5. Вычисление метрик:
        if args_task_ == 'regression':
            # Для регрессии:
            # Денормализуем предсказания и вычисляем MAE
            mae_error = mae(normalizer.denorm(output.data.cpu()), target)
            print('mae_error', mae_error)
            # Обновляем счетчики потерь и ошибок
            losses.update(loss.data.cpu(), target.size(0))
            mae_errors.update(mae_error, target.size(0))
        else:
            # Для классификации:
            # Вычисляем различные метрики качества
            accuracy, precision, recall, fscore, auc_score = \
                class_eval(output.data.cpu(), target)
            # Обновляем все счетчики метрик
            losses.update(loss.data.cpu().item(), target.size(0))
            accuracies.update(accuracy, target.size(0))
            precisions.update(precision, target.size(0))
            recalls.update(recall, target.size(0))
            fscores.update(fscore, target.size(0))
            auc_scores.update(auc_score, target.size(0))

        # print('Цикл 5. Вычисление метрик: завершен')

        # 6. Обратный проход (backward pass) и оптимизация:
        # Обнуляем градиенты (важно делать перед каждым backward)
        optimizer.zero_grad()
        # Вычисляем градиенты
        loss.backward()
        # Обновляем веса модели
        optimizer.step()

        # 7. Измеряем общее время обработки батча
        batch_time.update(time.time() - end)
        end = time.time()

        # 8. Периодический вывод статистики:
        if args_task_ == 'regression':
            # Вывод для регрессии
            print('Epoch: [{0}][{1}/{2}]\t'
                    'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                    'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                    'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                    'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
                epoch, i, len(train_loader), batch_time=batch_time,
                data_time=data_time, loss=losses, mae_errors=mae_errors)
            )

### пробуем один проход 

In [194]:
# способ для итерации 
batch = next(iter(train_loader))
batch

((tensor([[1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 

In [313]:
dataset

CIFData(559)

In [316]:
train_loader.dataset[0]

((tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 1., 0., 0., 0., 

In [317]:
val_loader.dataset[0]

((tensor([[0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 1., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0.,
           0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
           0., 0.],
          [0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
           0., 0., 0., 0., 0., 1., 0., 0., 0., 

In [195]:
# способ для итерации 
# batch = next(iter(train_loader))
batch

((tensor([[1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 

In [196]:
# данные о том какие элементы содержаться в материале
x = batch[0][0][0]
len(x), x, (x).shape, type(x)

(92,
 tensor([1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 1., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 0.,
         0., 0.]),
 torch.Size([92]),
 torch.Tensor)

In [197]:
batch[0][0].shape

torch.Size([847, 92])

In [198]:
batch[0][1].shape

torch.Size([847, 12, 41])

In [200]:
# данные о месторасположении атомов ???
x = batch[0][1][0][0]
len(x), x, (x).shape, type(x)

(41,
 tensor([0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 1.8483e-39,
         1.0847e-31, 8.6151e-25, 9.2603e-19, 1.3471e-13, 2.6521e-09, 7.0661e-06,
         2.5479e-03, 1.2434e-01, 8.2118e-01, 7.3397e-01, 8.8783e-02, 1.4534e-03,
         3.2201e-06, 9.6550e-10, 3.9178e-14, 2.1516e-19, 1.5991e-25, 1.6084e-32,
         2.1895e-40, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00,
         0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00]),
 torch.Size([41]),
 torch.Tensor)

In [201]:
x = batch[0][0]
len(x), x, (x).shape, type(x)

(847,
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]),
 torch.Size([847, 92]),
 torch.Tensor)

In [ ]:
# все входные данные кроме прогнозируемой величины
x = batch[0]
len(x), x, type(x)

(4,
 (tensor([[1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  .

In [212]:
# 32 значения прогнозируемой величины
x = batch[1]
len(x), x.shape, type(x), x  

(32,
 torch.Size([32, 1]),
 torch.Tensor,
 tensor([[-0.5033],
         [-0.3249],
         [-0.3679],
         [-0.4565],
         [-2.0655],
         [-0.5070],
         [-0.0283],
         [-0.0300],
         [ 0.6390],
         [-0.2140],
         [-1.9074],
         [-0.1738],
         [-3.3895],
         [-2.8427],
         [-0.2969],
         [-0.2082],
         [-0.8660],
         [-1.2103],
         [-1.5221],
         [-0.4448],
         [-1.2653],
         [-0.5825],
         [-0.2206],
         [-0.1846],
         [ 0.1920],
         [-1.6749],
         [-1.0399],
         [ 0.2659],
         [-2.3199],
         [ 0.7493],
         [-0.3112],
         [-0.1322]]))

In [214]:
# 32 значения номера файлов
x = batch[2]
len(x), type(x), x

(32,
 list,
 ['1191765',
  '961686',
  '642279',
  '636953',
  '19071',
  '865652',
  '1103513',
  '531',
  '631548',
  '1214552',
  '1211457',
  '861876',
  '4047',
  '1016821',
  '20211',
  '978526',
  '1200984',
  '1214037',
  '1364379',
  '867878',
  '754940',
  '510324',
  '1105932',
  '1227277',
  '1009813',
  '1186997',
  '1214132',
  '1008689',
  '555755',
  '999187',
  '972810',
  '14378'])

In [216]:
# 847 примеров структуры данные о элементах
x = batch[0][0]
len(x), x.shape, type(x), x

(847,
 torch.Size([847, 92]),
 torch.Tensor,
 tensor([[1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         [1., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]]))

In [ ]:
# 847 примеров структуры данные о ...
x = batch[0][1]
len(x), x.shape, type(x), x

(847,
 torch.Size([847, 12, 41]),
 torch.Tensor,
 tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]],
 
         ...,
 
         [[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],

In [ ]:
# 847х12 примеров структуры данные о ...
x = batch[0][2]
len(x), x.shape, type(x), x

(847,
 torch.Size([847, 12]),
 torch.Tensor,
 tensor([[ 19,  13,  16,  ...,   7,   9,   6],
         [ 14,  17,  11,  ...,   8,   9,   7],
         [ 18,  12,  15,  ...,   7,   9,   8],
         ...,
         [826, 816, 822,  ..., 835, 838, 837],
         [824, 823, 821,  ..., 836, 835, 833],
         [815, 810, 831,  ..., 835, 836, 838]]))

In [221]:
len(batch[0])

4

In [222]:
# 847 примеров структуры данные о ...
x = batch[0][3]
len(x), type(x), x

(32,
 list,
 [tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
          18, 19, 20, 21]),
  tensor([22, 23, 24]),
  tensor([25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42,
          43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53]),
  tensor([54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71,
          72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82]),
  tensor([83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96]),
  tensor([ 97,  98,  99, 100]),
  tensor([101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
          115]),
  tensor([116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
          130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143,
          144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157,
          158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169]),
  tensor([170, 171, 172, 173]),
  tensor([174, 175, 176, 177, 178, 17

In [173]:
len(next(iter(train_loader))[0]), next(iter(train_loader))[0], type(next(iter(train_loader))[0])

(4,
 (tensor([[1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          [1., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]),
  tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.]],
  
          [[0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           [0., 0., 0.,  ..., 0., 0., 0.],
           ...,
           [0., 0., 0.,  .

In [ ]:
# 2. Подготовка входных данных:

In [223]:
input = batch[0]

In [224]:
input_var = (
    Variable(input[0]),
    Variable(input[1]),
    input[2],
    input[3])

In [225]:
target = batch[1]

In [226]:
 # 3. Нормализация целевых значений:
target_normed = normalizer.norm(target)

In [228]:
target

tensor([[-0.5033],
        [-0.3249],
        [-0.3679],
        [-0.4565],
        [-2.0655],
        [-0.5070],
        [-0.0283],
        [-0.0300],
        [ 0.6390],
        [-0.2140],
        [-1.9074],
        [-0.1738],
        [-3.3895],
        [-2.8427],
        [-0.2969],
        [-0.2082],
        [-0.8660],
        [-1.2103],
        [-1.5221],
        [-0.4448],
        [-1.2653],
        [-0.5825],
        [-0.2206],
        [-0.1846],
        [ 0.1920],
        [-1.6749],
        [-1.0399],
        [ 0.2659],
        [-2.3199],
        [ 0.7493],
        [-0.3112],
        [-0.1322]])

In [231]:
target.max(), target.min()

(tensor(0.7493), tensor(-3.3895))

In [232]:
target_normed.max(), target_normed.min()

(tensor(1.4509), tensor(-2.5441))

In [227]:
target_normed

tensor([[ 0.2418],
        [ 0.4140],
        [ 0.3725],
        [ 0.2870],
        [-1.2661],
        [ 0.2383],
        [ 0.7003],
        [ 0.6987],
        [ 1.3444],
        [ 0.5211],
        [-1.1135],
        [ 0.5598],
        [-2.5441],
        [-2.0164],
        [ 0.4411],
        [ 0.5267],
        [-0.1083],
        [-0.4407],
        [-0.7416],
        [ 0.2983],
        [-0.4937],
        [ 0.1654],
        [ 0.5147],
        [ 0.5494],
        [ 0.9130],
        [-0.8890],
        [-0.2761],
        [ 0.9843],
        [-1.5117],
        [ 1.4509],
        [ 0.4272],
        [ 0.6001]])

In [233]:
args_cuda_

In [234]:
# Переносим целевые значения на GPU, если нужно
target_var = Variable(target_normed)

In [235]:
target_var

tensor([[ 0.2418],
        [ 0.4140],
        [ 0.3725],
        [ 0.2870],
        [-1.2661],
        [ 0.2383],
        [ 0.7003],
        [ 0.6987],
        [ 1.3444],
        [ 0.5211],
        [-1.1135],
        [ 0.5598],
        [-2.5441],
        [-2.0164],
        [ 0.4411],
        [ 0.5267],
        [-0.1083],
        [-0.4407],
        [-0.7416],
        [ 0.2983],
        [-0.4937],
        [ 0.1654],
        [ 0.5147],
        [ 0.5494],
        [ 0.9130],
        [-0.8890],
        [-0.2761],
        [ 0.9843],
        [-1.5117],
        [ 1.4509],
        [ 0.4272],
        [ 0.6001]])

In [236]:
# 4. Прямой проход (forward pass):
# Вычисляем выход модели для текущего батча
output = model(*input_var)
print('output', output)

output tensor([[-0.2288],
        [-0.2252],
        [-0.2218],
        [-0.2285],
        [-0.2247],
        [-0.2171],
        [-0.2265],
        [-0.2199],
        [-0.2182],
        [-0.2212],
        [-0.2042],
        [-0.2221],
        [-0.2281],
        [-0.2424],
        [-0.2163],
        [-0.2305],
        [-0.2168],
        [-0.2193],
        [-0.2262],
        [-0.2142],
        [-0.2408],
        [-0.2238],
        [-0.2321],
        [-0.2071],
        [-0.2268],
        [-0.2327],
        [-0.2421],
        [-0.2276],
        [-0.2224],
        [-0.2125],
        [-0.2389],
        [-0.2283]], grad_fn=<AddmmBackward0>)


In [237]:
# Вычисляем значение функции потерь
loss = criterion(output, target_var)
print('loss', loss)

loss tensor(0.9026, grad_fn=<MseLossBackward0>)


In [238]:
# Для регрессии:
# Денормализуем предсказания и вычисляем MAE
mae_error = mae(normalizer.denorm(output.data.cpu()), target)
print('mae_error', mae_error)

mae_error tensor(0.8435)


In [240]:
losses = AverageMeter()

In [ ]:
# Обновляем счетчики потерь и ошибок
losses.update(loss.data.cpu(), target.size(0))

In [244]:
loss.data, target.size(0)

(tensor(0.9026), 32)

In [245]:
mae_errors = AverageMeter()

In [ ]:
mae_errors.update(mae_error, target.size(0))

In [249]:
mae_errors.__dir__

<function AverageMeter.__dir__()>

In [250]:
dir(mae_errors)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'avg',
 'count',
 'reset',
 'sum',
 'update',
 'val']

In [253]:
mae_errors.__dict__

{'val': tensor(0.8435),
 'avg': tensor(0.8435),
 'sum': tensor(26.9923),
 'count': 32}

In [251]:
mae_errors.val

tensor(0.8435)

In [254]:
# 6. Обратный проход (backward pass) и оптимизация:
# Обнуляем градиенты (важно делать перед каждым backward)
optimizer.zero_grad()

In [255]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.01
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [256]:
# Вычисляем градиенты
loss.backward()

In [257]:
dir(loss)

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr_

In [259]:
loss

tensor(0.9026, grad_fn=<MseLossBackward0>)

In [258]:
loss.__dict__

{}

In [260]:
# Обновляем веса модели
optimizer.step()

In [261]:
optimizer

Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    capturable: False
    differentiable: False
    eps: 1e-08
    foreach: None
    fused: None
    initial_lr: 0.01
    lr: 0.01
    maximize: False
    weight_decay: 0
)

In [262]:
end = time.time()

In [264]:
batch_time = AverageMeter()

In [265]:
batch_time.update(time.time() - end)

In [267]:
batch_time.__dict__

{'val': 42.4198477268219,
 'avg': 42.4198477268219,
 'sum': 42.4198477268219,
 'count': 1}

In [272]:
from datetime  import datetime 

In [279]:
str(datetime.now())

'2025-04-09 17:04:02.975046'

In [280]:
data_time = datetime.now()

In [299]:
dir(data_time)

['__add__',
 '__class__',
 '__delattr__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__ne__',
 '__new__',
 '__radd__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rsub__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 'astimezone',
 'combine',
 'ctime',
 'date',
 'day',
 'dst',
 'fold',
 'fromisocalendar',
 'fromisoformat',
 'fromordinal',
 'fromtimestamp',
 'hour',
 'isocalendar',
 'isoformat',
 'isoweekday',
 'max',
 'microsecond',
 'min',
 'minute',
 'month',
 'now',
 'replace',
 'resolution',
 'second',
 'strftime',
 'strptime',
 'time',
 'timestamp',
 'timetuple',
 'timetz',
 'today',
 'toordinal',
 'tzinfo',
 'tzname',
 'utcfromtimestamp',
 'utcnow',
 'utcoffset',
 'utctimetuple',
 'weekday',
 'year']

In [302]:
data_time.date()

datetime.date(2025, 4, 9)

In [301]:
data_time.time()

datetime.time(17, 4, 36, 139024)

In [281]:
args_task_

'regression'

In [283]:
len(train_loader)

14

In [285]:
print(batch_time)

In [286]:
loss

tensor(0.9026, grad_fn=<MseLossBackward0>)

In [287]:
losses

In [288]:
mae_errors

In [295]:
batch_time.__dict__

{'val': 42.4198477268219,
 'avg': 42.4198477268219,
 'sum': 42.4198477268219,
 'count': 1}

In [296]:
batch_time.val

42.4198477268219

In [297]:
data_time

datetime.datetime(2025, 4, 9, 17, 4, 36, 139024)

In [291]:
print(f'Epoch: [{0}][{1}/{2}]\t'
            f'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t')

Epoch: [0][1/2]	Time 42.420 (42.420)	


In [308]:
loss

tensor(0.9026, grad_fn=<MseLossBackward0>)

In [306]:
dir(loss)

['H',
 'T',
 '__abs__',
 '__add__',
 '__and__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__complex__',
 '__contains__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__dlpack__',
 '__dlpack_device__',
 '__doc__',
 '__eq__',
 '__float__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__idiv__',
 '__ifloordiv__',
 '__ilshift__',
 '__imod__',
 '__imul__',
 '__index__',
 '__init__',
 '__init_subclass__',
 '__int__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__irshift__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__long__',
 '__lshift__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '__reduce_ex__',
 '__repr_

In [ ]:
# print(f'Loss {loss.val():.4f} ({loss:.4f})\t')

AttributeError: 'Tensor' object has no attribute 'val'

In [ ]:
# 8. Периодический вывод статистики:
if args_task_ == 'regression':
    # Вывод для регрессии
    print(f'Epoch: [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            # 'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
            'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
        epoch, i, len(train_loader), batch_time=batch_time,
        data_time=data_time, loss=losses, mae_errors=mae_errors)
    )

AttributeError: 'datetime.datetime' object has no attribute 'val'

In [310]:
# 8. Периодический вывод статистики:
if args_task_ == 'regression':
    # Вывод для регрессии
    print('Epoch: [{0}][{1}/{2}]\t'
            'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
            # 'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
            'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
            'MAE {mae_errors.val:.3f} ({mae_errors.avg:.3f})'.format(
        epoch, i, len(train_loader), batch_time=batch_time,
        data_time=data_time, loss=losses, mae_errors=mae_errors)
    )

Epoch: [59][3/14]	Time 42.420 (42.420)	Loss 0.9026 (0.9026)	MAE 0.844 (0.844)


### Обучаем модель на одной эпохе

In [ ]:
# for i, batch in enumerate(train_loader):
#     print(i, batch)

In [352]:
# Основной цикл обучения по батчам:
print('Начинаем цикл обучения по батчам!')
for i, (input, target, _) in tqdm(enumerate(train_loader)):
    print('Новый i-й шаг цикла:', i)
    print('input - входные данные модели', input)
    print('target - целевые значения', target)

Начинаем цикл обучения по батчам!


4it [00:00, 31.01it/s]

Новый i-й шаг цикла: 0
input - входные данные модели (tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]],

        [[0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         ...,
         [0., 0., 0.,  ..., 0.

8it [00:00, 26.12it/s]

Новый i-й шаг цикла: 6
input - входные данные модели (tensor([[1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        [1., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e

14it [00:00, 24.29it/s]

Новый i-й шаг цикла: 10
input - входные данные модели (tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]]), tensor([[[1.4013e-45, 4.6788e-37, 1.5139e-29,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [1.4013e-45, 4.6788e-37, 1.5139e-29,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 2.0336e-41,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         ...,
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00],
         [0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00,
          0.0000e+00, 0.0000e+00]],

        [[1.4013e-45, 4.6788e-37, 1.5139e-29,  ..., 0.0000e+00,
          0.0000

In [357]:
epoch = 0
# Обучаем модель на одной эпохе
train(train_loader, model, criterion, optimizer, epoch, normalizer)

batch_time <__main__.AverageMeter object at 0x0000013FA0EF1460>
data_time <__main__.AverageMeter object at 0x0000013FA107F290>
losses <__main__.AverageMeter object at 0x0000013FA0EF16A0>
mae_errors <__main__.AverageMeter object at 0x0000013FA0EF1790>
Сработало model.train()
end 1744232365.7225332
Начинаем цикл обучения по батчам!


0it [00:00, ?it/s]

Новый i-й шаг цикла: 0
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[5.9803],
        [4.8924],
        [5.3085],
        [5.1622],
        [4.9581],
        [4.8454],
        [7.0328],
        [4.6454],
        [4.8978],
        [4.8939],
        [5.2195],
        [4.8798],
        [4.8690],
        [4.5000],
        [4.6881],
        [5.3167],
        [4.6732],
        [4.6934],
        [4.6144],
        [4.5171],
        [4.9836],
        [4.5448],
        [4.7325],
        [4.6509],
        [4.4760],
        [4.8249],
        [4.5047],
        [4.8036],
        [4.6441],
        [4.5598],
        [4.7957],
        [4.7079]], grad_fn=<AddmmBackward0>)
loss tensor(24.1384, grad_fn=<MseLossBackward0>)
mae_error tensor(5.0138)


2it [00:00,  3.09it/s]

Epoch: [0][0/14]	Time 0.523 (0.523)	Data 0.007 (0.007)	Loss 24.1384 (24.1384)	MAE 5.014 (5.014)
Новый i-й шаг цикла: 1
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[1.0437],
        [1.0089],
        [1.0068],
        [1.0576],
        [1.0484],
        [1.0058],
        [1.0417],
        [1.0492],
        [1.0757],
        [1.0322],
        [1.0351],
        [1.0028],
        [1.0215],
        [0.9932],
        [1.0251],
        [1.0803],
        [1.0400],
        [1.0359],
        [0.9926],
        [1.0199],
        [1.0323],
        [1.0432],
        [1.0399],
        [1.0397],
        [0.9468],
        [1.0483],
        [1.0364],
        [1.0473],
        [1.0376],
        [1.0995],
        [1.0323],
        [1.0185]], grad_fn=<AddmmBackward0>)
loss tensor(1.2440, grad_fn=<MseLossBackward0>)
mae_error tensor(0.8661)
Epoch: [0][1/14]	Time 0.186 (0.355)	Data 0.004 (0.005)	Loss 1.2440 (12.6912)

...

Epoch: [0][10/14]	Time 0.169 (0.243)	Data 0.003 (0.004)	Loss 1.2402 (3.5249)	MAE 0.878 (1.432)
Новый i-й шаг цикла: 11
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[0.2234],
        [0.2247],
        [0.2247],
        [0.2249],
        [0.2198],
        [0.2140],
        [0.2244],
        [0.2246],
        [0.2241],
        [0.2245],
        [0.2244],
        [0.2247],
        [0.2133],
        [0.2239],
        [0.2246],
        [0.2117],
        [0.2247],
        [0.2250],
        [0.2121],
        [0.2222],
        [0.2253],
        [0.2164],
        [0.2235],
        [0.2145],
        [0.2197],
        [0.2123],
        [0.2234],
        [0.2236],
        [0.2239],
        [0.2223],
        [0.2252],
        [0.2251]], grad_fn=<AddmmBackward0>)
loss tensor(0.9922, grad_fn=<MseLossBackward0>)
mae_error tensor(0.6870)
Epoch: [0][11/14]	Time 0.204 (0.239)	Data 0.003 (0.004)	Loss 0.9922 (3.3139)

13it [00:03,  4.91it/s]

Новый i-й шаг цикла: 12
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[0.3145],
        [0.3125],
        [0.3130],
        [0.3132],
        [0.3079],
        [0.3063],
        [0.3130],
        [0.3109],
        [0.3133],
        [0.3134],
        [0.3121],
        [0.3122],
        [0.3144],
        [0.3031],
        [0.3116],
        [0.3106],
        [0.3144],
        [0.3091],
        [0.3041],
        [0.3137],
        [0.3133],
        [0.3144],
        [0.3107],
        [0.3125],
        [0.3129],
        [0.3104],
        [0.3071],
        [0.3111],
        [0.3080],
        [0.3110],
        [0.3138],
        [0.3120]], grad_fn=<AddmmBackward0>)
loss tensor(0.5715, grad_fn=<MseLossBackward0>)
mae_error tensor(0.5825)
Epoch: [0][12/14]	Time 0.204 (0.237)	Data 0.004 (0.004)	Loss 0.5715 (3.1029)	MAE 0.583 (1.310)


14it [00:03,  4.32it/s]

Новый i-й шаг цикла: 13
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[0.3785],
        [0.3791],
        [0.3783],
        [0.3720],
        [0.3787],
        [0.3730],
        [0.3790],
        [0.3722],
        [0.3783],
        [0.3760],
        [0.3792],
        [0.3775],
        [0.3783],
        [0.3776],
        [0.3711],
        [0.3790],
        [0.3730],
        [0.3785],
        [0.3797],
        [0.3799],
        [0.3773],
        [0.3796],
        [0.3800],
        [0.3801],
        [0.3717],
        [0.3801],
        [0.3792],
        [0.3795],
        [0.3726],
        [0.3780],
        [0.3789]], grad_fn=<AddmmBackward0>)
loss tensor(1.2570, grad_fn=<MseLossBackward0>)
mae_error tensor(0.7436)
Epoch: [0][13/14]	Time 0.162 (0.231)	Data 0.004 (0.004)	Loss 1.2570 (2.9749)	MAE 0.744 (1.270)


In [350]:
args_start_epoch_

0

In [351]:
args_epochs_

60

### Запускаем цикл обучения по эпохам

In [ ]:
# задаём очередную глобальную переменную
best_mae_error = 1e10
best_mae_error

10000000000.0

In [367]:
# 7. ЦИКЛ ОБУЧЕНИЯ
for epoch in tqdm(range(args_start_epoch_, args_epochs_)):
    # Обучаем модель на одной эпохе
    train(train_loader, model, criterion, optimizer, epoch, normalizer)

    # Оцениваем на валидационной выборке
    mae_error = validate(val_loader, model, criterion, normalizer)

    # Проверяем на NaN (если ошибка NaN, прекращаем выполнение)
    if mae_error != mae_error:
        print('Exit due to NaN')
        sys.exit(1)

    # Обновляем скорость обучения
    scheduler.step()

    # Сохраняем лучшую модель
    if args_task_ == 'regression':
        is_best = mae_error < best_mae_error  # для регрессии ищем минимум ошибки
        best_mae_error = min(mae_error, best_mae_error)
    else:
        is_best = mae_error > best_mae_error  # для классификации может быть другой критерий
        best_mae_error = max(mae_error, best_mae_error)
    
    # Сохраняем чекпойнт
    save_checkpoint({
        'epoch': epoch + 1,          # текущая эпоха
        'state_dict': model.state_dict(),  # параметры модели
        'best_mae_error': best_mae_error,  # лучшая ошибка
        'optimizer': optimizer.state_dict(),  # состояние оптимизатора
        'normalizer': normalizer.state_dict(),  # параметры нормализатора
        # 'args': vars(args)           # аргументы командной строки
    }, is_best)  # is_best указывает, является ли это лучшей моделью

  0%|          | 0/60 [00:00<?, ?it/s]

batch_time <__main__.AverageMeter object at 0x0000013FA0EF20C0>
data_time <__main__.AverageMeter object at 0x0000013FA0EB0E30>
losses <__main__.AverageMeter object at 0x0000013FA0F2C230>
mae_errors <__main__.AverageMeter object at 0x0000013FA0F2D6D0>
Сработало model.train()
end 1744266363.297792
Начинаем цикл обучения по батчам!


Новый i-й шаг цикла: 0
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[-0.2052],
        [ 0.1319],
        [-0.0107],
        [ 0.2347],
        [ 0.2746],
        [ 0.1151],
        [ 0.6848],
        [-1.2000],
        [-0.3585],
        [ 0.7763],
        [-0.1774],
        [ 0.2884],
        [-0.7136],
        [-0.2179],
        [-0.1926],
        [-1.2658],
        [-1.3840],
        [-0.1437],
        [-0.0960],
        [ 0.1244],
        [-0.0809],
        [-0.2195],
        [-0.0784],
        [-0.1054],
        [ 0.2348],
        [-0.2154],
        [-1.3193],
        [ 0.0107],
        [ 0.1985],
        [ 0.2190],
        [ 0.6247],
        [-0.2351]], grad_fn=<AddmmBackward0>)
loss tensor(0.2367, grad_fn=<MseLossBackward0>)
mae_error tensor(0.4281)


Epoch: [0][0/14]	Time 0.342 (0.342)	Data 0.012 (0.012)	Loss 0.2367 (0.2367)	MAE 0.428 (0.428)
Новый i-й шаг цикла: 1
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[-1.5091],
        [-1.4985],
        [-0.0759],
        [-0.2094],
        [-1.2633],
        [-0.0860],
        [ 0.1684],
        [-1.5094],
        [ 0.3489],
        [ 0.3491],
        [-0.3857],
        [-0.1266],
        [-0.0458],
        [-0.2091],
        [-0.1479],
        [-1.2770],
        [-0.2802],
        [-0.3733],
        [-0.1569],
        [-0.0651],
        [ 0.0141],
        [-0.2624],
        [ 0.4585],
        [ 0.3351],
        [-0.1813],
        [-1.5971],
        [-0.3492],
        [-1.4955],
        [-0.2228],
        [ 0.0321],
        [-1.1285],
        [ 0.1870]], grad_fn=<AddmmBackward0>)
loss tensor(0.4291, grad_fn=<MseLossBackward0>)
mae_error tensor(0.5973)


Epoch: [0][1/14]	Time 0.296 (0.319)	Data 0.011 (0.011)	Loss 0.4291 (0.3329)	MAE 0.597 (0.513)
Новый i-й шаг цикла: 2
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[-4.4533e-01],
        [-1.2511e+00],
        [ 1.0095e-01],
        [-3.5268e-01],
        [-5.1134e-02],
        [-1.6084e-01],
        [ 2.5603e-01],
        [-1.4535e+00],
        [ 1.7632e-01],
        [ 1.2707e-01],
        [-4.2042e-01],
        [ 1.0549e-02],
        [-1.4326e+00],
        [-1.3416e+00],
        [-3.6845e-01],
        [ 2.9250e-01],
        [-4.4762e-01],
        [-1.1469e+00],
        [-3.6037e-01],
        [-6.2644e-01],
        [-1.4793e+00],
        [-4.7666e-01],
        [-3.3865e-01],
        [-1.5420e+00],
        [ 2.5893e-02],
        [-1.0710e-01],
        [-4.0249e-01],
        [ 1.0761e-01],
        [-1.0215e+00],
        [-4.4063e-01],
        [-2.3898e-01],
        [-1.1574e-03]], grad_fn=<AddmmBac

...

Epoch: [59][11/14]	Time 0.436 (0.310)	Data 0.012 (0.010)	Loss 0.0411 (0.0568)	MAE 0.157 (0.185)
Новый i-й шаг цикла: 12
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[-0.9915],
        [ 0.4253],
        [ 1.3667],
        [ 0.5188],
        [ 0.6153],
        [ 0.3250],
        [ 0.5548],
        [ 0.3644],
        [ 0.6314],
        [ 0.0170],
        [ 0.6712],
        [-2.0690],
        [ 0.4776],
        [ 0.5719],
        [ 0.5746],
        [ 0.4114],
        [ 0.5170],
        [-0.0611],
        [ 0.2521],
        [ 0.2972],
        [-0.0711],
        [ 0.2105],
        [ 0.1754],
        [ 0.5533],
        [ 0.3089],
        [-1.9847],
        [ 0.3512],
        [-0.3072],
        [ 0.6314],
        [ 0.3677],
        [ 0.2622],
        [ 0.3159]], grad_fn=<AddmmBackward0>)
loss tensor(0.0293, grad_fn=<MseLossBackward0>)
mae_error tensor(0.1433)


Epoch: [59][12/14]	Time 0.256 (0.305)	Data 0.011 (0.010)	Loss 0.0293 (0.0547)	MAE 0.143 (0.182)
Новый i-й шаг цикла: 13
Измеряем время загрузки данных
args_cuda_ else
Нормализация целевых значений: args_task_ == regression
args_cuda_ is None
output tensor([[ 0.4577],
        [ 0.6867],
        [-1.7396],
        [ 0.1304],
        [ 0.5323],
        [ 0.4289],
        [ 0.4331],
        [ 0.1730],
        [-1.5280],
        [-0.4079],
        [ 0.7299],
        [ 0.2764],
        [ 1.6671],
        [-2.1636],
        [ 0.0162],
        [ 0.4555],
        [ 1.0753],
        [ 0.7697],
        [ 0.1613],
        [-1.9710],
        [ 1.7846],
        [ 0.7318],
        [-2.1955],
        [ 0.6323],
        [ 0.8819],
        [-1.5351],
        [-2.1841],
        [ 0.5189],
        [-1.5369],
        [ 0.6835],
        [ 0.3449]], grad_fn=<AddmmBackward0>)
loss tensor(0.0321, grad_fn=<MseLossBackward0>)
mae_error tensor(0.1502)


14it [00:04,  3.11it/s]
100%|██████████| 60/60 [04:54<00:00,  4.77s/it]

Epoch: [59][13/14]	Time 0.528 (0.321)	Data 0.012 (0.010)	Loss 0.0321 (0.0531)	MAE 0.150 (0.179)
Test: [0/2]	Time 0.084 (0.084)	Loss 0.1063 (0.1063)	MAE 0.239 (0.239)
 * MAE 0.231


100%|██████████| 60/60 [04:54<00:00,  4.91s/it]


### 8. ТЕСТИРОВАНИЕ ЛУЧШЕЙ МОДЕЛИ

In [368]:
# 8. ТЕСТИРОВАНИЕ ЛУЧШЕЙ МОДЕЛИ
print('---------Evaluate Model on Test Set---------------')

---------Evaluate Model on Test Set---------------


In [369]:
# Загружаем лучшую модель
best_checkpoint = torch.load('model_best.pth.tar')

In [371]:
type(best_checkpoint)

dict

In [373]:
best_checkpoint.keys()

dict_keys(['epoch', 'state_dict', 'best_mae_error', 'optimizer', 'normalizer'])

In [376]:
best_checkpoint['epoch']

60

In [378]:
best_checkpoint['best_mae_error']

tensor(0.2305)

In [380]:
best_checkpoint['normalizer']

{'mean': tensor(-0.7538), 'std': tensor(1.0360)}

In [379]:
best_checkpoint['optimizer']

{'state': {0: {'step': tensor(897.),
   'exp_avg': tensor([[-1.1318e-04,  2.7945e-04, -2.1355e-04,  ..., -6.8079e-05,
            -3.3205e-05,  3.2170e-05],
           [-3.6995e-05,  3.8910e-04,  2.2017e-04,  ..., -1.2276e-04,
             5.8465e-05,  2.5712e-04],
           [-6.7273e-05,  4.6766e-04, -3.4744e-05,  ..., -1.6269e-04,
             6.0009e-05,  8.3028e-05],
           ...,
           [ 2.6039e-05, -3.0643e-04,  2.1405e-04,  ...,  2.3504e-05,
            -3.7679e-05, -5.0204e-05],
           [ 1.5025e-04, -2.2935e-04,  1.4358e-04,  ...,  1.5376e-04,
            -4.8349e-05, -6.3108e-05],
           [-2.3565e-06,  6.3680e-04, -8.2482e-04,  ..., -2.8508e-04,
            -1.6004e-06,  1.2231e-04]]),
   'exp_avg_sq': tensor([[1.3710e-06, 1.3273e-06, 4.0602e-06,  ..., 2.3030e-06, 3.3281e-07,
            3.0924e-07],
           [9.6210e-07, 1.7936e-06, 2.9316e-06,  ..., 2.7606e-06, 4.2082e-07,
            3.6264e-07],
           [1.4552e-06, 1.8933e-06, 3.4349e-06,  ..., 1.6527

In [370]:
best_checkpoint

{'epoch': 60,
 'state_dict': OrderedDict([('embedding.weight',
               tensor([[ 0.0889, -0.1733, -0.0197,  ...,  0.1453, -0.0216,  0.0817],
                       [ 0.0858,  0.0132, -0.1028,  ..., -0.0861, -0.1155,  0.1273],
                       [-0.1013, -0.2309,  0.0049,  ...,  0.2026, -0.2213,  0.0497],
                       ...,
                       [-0.0878,  0.1290, -0.0052,  ..., -0.0441, -0.0177, -0.1694],
                       [ 0.1478,  0.0199,  0.1295,  ...,  0.0147, -0.0239, -0.0753],
                       [ 0.0364, -0.0497,  0.0703,  ...,  0.1500, -0.0797, -0.0201]])),
              ('embedding.bias',
               tensor([-0.0520,  0.1922, -0.2080, -0.0309,  0.0285, -0.0373, -0.0920, -0.1509,
                       -0.0456, -0.2784, -0.0371, -0.2041, -0.0786,  0.0937, -0.1163, -0.0098,
                       -0.0219, -0.0391,  0.0417,  0.0793,  0.2666, -0.2090, -0.0318, -0.1613,
                        0.3446, -0.1606, -0.0069,  0.1555, -0.1163,  0.1512, -

In [381]:
model.load_state_dict(best_checkpoint['state_dict'])

<All keys matched successfully>

In [382]:
# Оцениваем на тестовой выборке
validate(test_loader, model, criterion, normalizer, test=True)

Test: [0/2]	Time 0.184 (0.184)	Loss 0.2652 (0.2652)	MAE 0.320 (0.320)
 ** MAE 0.273


tensor(0.2733)

## Вывод